In [1]:
%load_ext autoreload 
%autoreload 2
%reload_ext autoreload

import xarray as xr
import numpy as np
import os
import pandas as pd
from glob import glob
import datetime as dt
from PIL import Image
import sys
import functions as f
import pickle
import tensorflow as tf
from keras.optimizers import Adam
from keras.callbacks import ReduceLROnPlateau
from keras.models import Model
from keras import backend as K
from tensorflow.keras.utils import plot_model
from IPython.display import Image 
from tensorflow.python.data.ops import dataset_ops
import dask
from keras.models import load_model
from sklearn.metrics import mean_squared_error
from io import StringIO
import shap
from keras.layers import Input
import keras as k
from contextlib import redirect_stdout
from PIL import Image
import matplotlib.gridspec as gridspec
import cv2
import channelExperiment as CE
import loadDataWeek0 as loadData
import bottleneck as bn
from tensorflow.keras import mixed_precision
import csv
import addPredictors as pred
import loadValues as lv
import masks
import denseValue
import verifications
import preprocessUtils as putils
import matplotlib.pyplot as plt

global lead_week_predictions 
lead_week_predictions = [1,2,3,4,5] #for running the model for different lead weeks. Can choose any numbers in a LIST from [1,2,3,4,5]


global region_name,testing_scenario,save_loss_name
region_name = 'CONUS' #['australia','CONUS', 'china']


    
'''Make sure to fill in this information depending on what test we are doing'''
make_single_prediction_from_model_for_testing=False
make_additional_predictions_from_model_for_testing = False
permutation_test_graphs_create=True
bias_correction_predict = False

#Tensorflow RT things
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)
tf.config.optimizer.set_jit(True) # Enable XLA.


global RZSM_or_Tmax_or_both, num_predictions_testing, num_train_val_predictions
RZSM_or_Tmax_or_both = 'RZSM' # for getting the predictor from either RZSM and Tmax ('both') or only RZSM ('RZSM')

num_predictions_testing = 25
num_train_val_predictions=10

#for permutation plot
max_RZSM_value,max_tmax_value = 0.05,5

#set for the larger memeory ones
# os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'


global ref_source,final_testing_year, test_year
ref_source = 'GEFSv12'
final_testing_year = 2019
test_year = final_testing_year

2024-08-14 15:23:06.098873: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-14 15:23:46.088428: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/glade/work/klesinger/conda-envs/tf212gpu_new/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Your GPU may run slowly with dtype policy mixed_float16 because it does not have compute capability of at least 7.0. Your GPU:
  Quadro GP100, compute capability 6.0
See https://developer.nvidia.com/cuda-gpus for a list of GPUs and their compute capabilities.
If you will use compatible GPU(s) not attached to this host, e.g. by running a multi-worker model, you can ignore this warning. This message will only be logged once


# Choose different loss and architecture configurations

In [2]:

#First just re-save the new density value into a new file (it's convoluted but it works)
# denseValue.return_dense_value_file(region_name, 0.5) #can chooose 0.5 or 0.8 as the alpha coefficient


import losses

testing_scenario = 'regular' #['dense', 'regular', 'transformer', 'super_pixel', 'attention', 'denseLarge', 'basic', 'regularResidual','regularDepth']

global save_loss_name, loss_fn

if testing_scenario == 'dense':
    import modelRzsmDenseRelu as UNETRzsm
    
    save_loss_name = 'denseLoss'
    loss_fn = losses.crps2d_tf_dense_test2
    
elif testing_scenario == 'regular':
    import modelRzsmReluExtraConv as UNETRzsm

    save_loss_name = 'regular'
    loss_fn = losses.crps2d_tf

elif testing_scenario == 'regularDepth':
    import modelRzsmReluExtraConvDepthwise as UNETRzsm

    save_loss_name = 'regularDepth'
    loss_fn = losses.crps2d_tf

elif testing_scenario == 'transformer':
    import modelRzsmReluTransformer as UNETRzsm
    
    save_loss_name = 'transformer'
    loss_fn = losses.crps2d_tf_dense_test2

elif testing_scenario == 'super_pixel':
    import modelRzsmReluPixelSuper as UNETRzsm
    
    save_loss_name = 'supPixel'
    loss_fn = losses.crps2d_tf_dense_test2

elif testing_scenario == 'attention':
    import modelRzsmReluAttention as UNETRzsm
    
    save_loss_name = 'attention'
    loss_fn = losses.crps2d_tf_dense_test2
    
elif testing_scenario == 'None':
    import modelRzsm2DdropoutRelu as UNETRzsm
    
    save_loss_name = ''
    loss_fn = losses.crps2d_tf

elif testing_scenario == 'denseLarge':
    import modelRzsmReluExtraConv as UNETRzsm

    save_loss_name = 'denseLarge'
    loss_fn = losses.crps2d_tf_dense_test2
    
elif testing_scenario == 'basic':
    import basicUNET as UNETRzsm

    save_loss_name = testing_scenario
    loss_fn = losses.crps2d_tf

elif testing_scenario == 'regularResidual':
    #For predicting residuals
    import modelRzsmReluExtraConvResidual as UNETRzsm

    save_loss_name = 'regularResidual'
    loss_fn = losses.crps2d_tf

2024-08-14 15:26:22.458212: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15448 MB memory:  -> device: 0, name: Quadro GP100, pci bus id: 0000:18:00.0, compute capability: 6.0


Using TensorFlow backend


# This script will load data based on specific lead times that we want to experiment with and what Experiments we choose to run

# Week 1 (lead index 6 of the forecast)

| Ex. Number/Name | Obs. RZSM Inputs | Obs. Other Inputs                    | UNET predicted Inputs (N>=2)     |Reforecast Inputs| Prediction Lead Week/Variables | Activation function | Loss function | Batch size |
| ----------------| -----------------| ------------------------------------ | -------------------      |--------          | -------------------| ---------------------| ------------------------| ------------|
| 0 - EX0         | None             | None                                 | None                     | RZSM, Wk 1-N| Wk N     -  RZSM |              Relu             | CRPS experimental| 66
| 1 - EX1         | Wk. lags 1-3    | pwat,spfh,tmax,z200,diff_temp lags 1-3 | RZSM Wk 1-N |        None |  Wk N     -  RZSM |             Relu             | CRPS experimental | 66
| 2 - EX2         | Wk. lags 1-6   | pwat,spfh,tmax,z200,diff_temp lags 1-3  | RZSM Wk 1-N          |  None |  Wk N     -  RZSM |       Relu             | CRPS experimental| 66
| 3 - EX3         | Wk. lags 1-9    | pwat,spfh,tmax,z200,diff_temp lags 1-3 | RZSM Wk 1-N         |  None |  Wk N     -  RZSM |       Relu             | CRPS experimental| 66
| 4 - EX4         | Wk. lags 1-12    | pwat,spfh,tmax,z200,diff_temp lags 1-3| RZSM Wk 1-N         |  None |  Wk N     -  RZSM |      Relu             | CRPS experimental| 66
| 5 - EX5         | Wk. lags 1-3   | None                                   | None                   | RZSM, week N | Wk N     -  RZSM  |         Relu             | CRPS experimental| 66
| 6 - EX6         | Wk. lags 1-6   | None                                     | None                   |RZSM, week N  | Wk N     -  RZSM  |        Relu             | CRPS experimental| 66
| 7 - EX7         | Wk. lags 1-9    | None                                | None                   |RZSM, week N  | Wk N     -  RZSM  |         Relu             | CRPS experimental |66
| 8 - EX8         | Wk. lags 1-12    | None                                  | None                   |RZSM, week N  | Wk N     -  RZSM |         Relu             | CRPS experimental | 66
| 9 - EX9         | Wk. lags 1-3    |pwat,spfh,tmax,z200,diff_temp lags 1-3  | RZSM Wk 1-N                  |RZSM  week N  | Wk N     -  RZSM  |         Relu             | CRPS experimental| 66
| 10 - EX10       | Wk. lags 1-6    |pwat,spfh,tmax,z200,diff_temp  lags 1-3 | RZSM Wk 1-N                  |RZSM  week N  |Wk N     -  RZSM   |         Relu             | CRPS experimental| 66
| 11 - EX11       | Wk. lags 1-9    |pwat,spfh,tmax,z200,diff_temp  lags 1-3 | RZSM Wk 1-N                  |RZSM  week N  | Wk N     -  RZSM  |         Relu             | CRPS experimental| 66
| 12 - EX12       | Wk. lags 1-12   |pwat,spfh,tmax,z200,diff_temp lags 1-3  | RZSM Wk 1-N                  |RZSM  week N  | Wk N     -  RZSM  |         Relu             | CRPS experimental| 66
| 13 - EX13       | None             | None                                 | None                     | RZSM week N| Wk N     -  RZSM |              Relu             | CRPS experimental| 66
| 14 - EX14         | Wk. lags 1-3    | pwat,spfh,tmax,z200,diff_temp lags 1-3 | None |        None |  Wk N     -  RZSM |             Relu             | CRPS experimental | 66
| 15 - EX15         | Wk. lags 1-6   | pwat,spfh,tmax,z200,diff_temp lags 1-3  | None             |  None |  Wk N     -  RZSM |       Relu             | CRPS experimental| 66
| 16 - EX16         | Wk. lags 1-9    | pwat,spfh,tmax,z200,diff_temp lags 1-3 | None            |  None|  Wk N     -  RZSM |       Relu             | CRPS experimental| 66
| 17 - EX17        | Wk. lags 1-12    | pwat,spfh,tmax,z200,diff_temp lags 1-3| None            |  None|  Wk N     -  RZSM |      Relu             | CRPS experimental| 66
| 18 - EX18         | Wk. lags 1-3   | None                                   | RZSM Wk 1-N                 | RZSM, week N | Wk N     -  RZSM  |         Relu             | CRPS experimental| 66
| 19 - EX19         | Wk. lags 1-6   | None                                     | RZSM Wk 1-N                  |RZSM, week N  | Wk N     -  RZSM  |        Relu             | CRPS experimental| 66
| 20 - EX20         | Wk. lags 1-9    | None                                | RZSM Wk 1-N                  |RZSM, week N  | Wk N     -  RZSM  |         Relu             | CRPS experimental |66
| 21 - EX21         | Wk. lags 1-12    | None                                  | RZSM Wk 1-N                   |RZSM, week N  | Wk N     -  RZSM |         Relu             | CRPS experimental | 66
| 22 - EX22         | Wk. lags 1-3   | None                                   | None                   | None | Wk N     -  RZSM  |         Relu             | CRPS experimental| 66
| 23 - EX23         | Wk. lags 1-6   | None                                     | None                   |None  | Wk N     -  RZSM  |        Relu             | CRPS experimental| 66
| 24 - EX24         | Wk. lags 1-9    | None                                | None                   |None  | Wk N     -  RZSM  |         Relu             | CRPS experimental |66
| 25 - EX25         | Wk. lags 1-12    | None                                  | None                   |None  | Wk N     -  RZSM |         Relu             | CRPS experimental | 66
| 26 - EX26         | None    | None                                  | None                   |Wk 1-N (choose best models)  | Wk N     -  RZSM |         Relu             | CRPS experimental | 66
| 27 - EX27         | Wk. lags 1-3    |pwat,spfh,tmax,z200,diff_temp lags 1-3  |  None                   |Wk1 RZSM,tmax, diff_temp, z200, pwat, spfh  | Wk N     -  RZSM  |         Relu             | CRPS experimental| 66 (like EX9)
| 28 - EX28         | Wk. lags 1-6    |pwat,spfh,tmax,z200,diff_temp lags 1-3  |  None                   |Wk1 RZSM,tmax, diff_temp, z200, pwat, spfh  | Wk N     -  RZSM  |         Relu             | CRPS experimental| 66 (like EX10)

# EX 1-4 - Testing if the prediction of previous week adds value (but don't include the current week that is trying to be predicted). Observation driven but with a prediction.
# EX 14-17 - Testing if the prediction of previous week adds value (but don't include the current week that is trying to be predicted). Purely observation driven.
# EX 5-8 - Testing if adding other observations (pwat, spfh, etc) has an increase or decrease in skill.
# EX 18-21 - Seeing if adding the previous week gains additional skill
# EX 22-25 - Purely observation driven. Only soil moisture.

In [3]:
input_directory = f'Data/model_npy_inputs/{region_name}/Model_input_data' #For inputs into the model
verification_directory = f'Data/model_npy_inputs/{region_name}/Verification_data' #For observation verification
start_of_forecast_init = '2000-01-05'
end_of_forecast_init = '2019-12-25'

global training_size_shape
training_size_shape = np.array((9185,48,96))

global validation_testing_size_shape
validation_testing_size_shape = np.array((1144,48,96))

'''This decides how many lag weeks we have for data such as pwat, z200, tmax, etc'''
global observation_lag_list_not_RZSM
#This is for observations pwat, z200, spfh, tmax, diff_temp variables used as predictors
#These are the day lags which were already computed as the 7-day rolling mean

observation_lag_list_not_RZSM = [-1,-7,-14] 

global train_start, train_end, val_start, val_end, test_start, test_end
train_start = '2000-01-01'
train_end = '2015-12-31'

val_start = '2016-01-01'
val_end = '2017-12-31'

test_start = '2018-01-01'
test_end = '2019-12-31'


In [4]:
def load_all_data_EX0(lead, num_lags_obs_RZSM, include_lags_obs_pwat_spfh_tmax, include_reforecast_or_not, observation_lag_list_not_RZSM, lag_integer_list, input_directory,training_size_shape,validation_testing_size_shape,RZSM_or_Tmax_or_both,addtl_experiment,experiment_test):

    soil_var = 'soilw_bgrnd' #This is the observation verification variable name
    
    channel_list = []

    if addtl_experiment == False:
        assert lead >=1, 'Lead must be >=1, do not look at Week 0'
        
        lead_list = np.arange(1,lead+1)

        print(f'Only adding Reforecast data as input for leads {list(lead_list)}.')

        training_input = np.empty(shape = (training_size_shape[0],training_size_shape[1],training_size_shape[2],lead)) #We are adding both RZSM and/or Tmax (so use * 2)
        validation_input = np.empty(shape = (validation_testing_size_shape[0],validation_testing_size_shape[1],validation_testing_size_shape[2],lead))
        testing_input = np.empty(shape = (validation_testing_size_shape[0],validation_testing_size_shape[1],validation_testing_size_shape[2],lead))

        #Add reforecast data
        for idx,lead in enumerate(lead_list):
            channel_list.append(f'RZSM_ref_lead{lead}')

            training_input, validation_input, testing_input = pred.add_reforecast_by_lag(training_input, validation_input, testing_input, lead, input_directory,soil_var, idx,ref_source,final_testing_year)

        print(f'Index idx value is {idx}. Done adding RZSM obs.')    

    
    elif addtl_experiment == True:
        print(f'Only adding Reforecast data as input for lead {lead}.')

        lead_add = 1 #We are only adding the current week of reforecast
        lead_list = [lead]

        training_input = np.empty(shape = (training_size_shape[0],training_size_shape[1],training_size_shape[2],lead_add)) #We are adding both RZSM and Tmax (so use * 2)
        validation_input = np.empty(shape = (validation_testing_size_shape[0],validation_testing_size_shape[1],validation_testing_size_shape[2],lead_add))
        testing_input = np.empty(shape = (validation_testing_size_shape[0],validation_testing_size_shape[1],validation_testing_size_shape[2],lead_add))

        channel_list.append(f'RZSM_ref_lead{lead}')

        idx = 0
        training_input, validation_input, testing_input = pred.add_reforecast_by_lag(training_input, validation_input, testing_input, lead, input_directory,soil_var, idx,ref_source,final_testing_year)

    return(training_input, validation_input, testing_input, channel_list)

In [5]:
def load_all_data_EX1_EX2_EX3_EX4_after_week_0(lead, num_lags_obs_RZSM, include_lags_obs_pwat_spfh_tmax, include_reforecast_or_not, 
                                           observation_lag_list_not_RZSM, lag_integer_list, input_directory,training_size_shape,
                                           validation_testing_size_shape,experiment_name,RZSM_or_Tmax_or_both,addtl_experiment,experiment_test, 
                                               experiment_name_out,ref_source,final_testing_year):
    channel_list = []
    
    #Reforecast predictors
    var_list = ['pwat_eatm', 'spfh_2m', 'tmax_2m', 'diff_temp_2m', 'hgt_pres']

    assert lead >=1, 'Lead must be >=1, we are not testing week 0 anymore'

    if addtl_experiment == False:
        #Get the number of extra channels to add from previous weeks predictions
        if RZSM_or_Tmax_or_both == 'both':
            add_channels = lead * 2
        else:
            #Also add the number of channels according to the lead. If lead == 1, then only add lead 1 current week. If lead == 2, add one channel for week 2 and one channel for the prediction from Week 1
            if lead > 1:
                add_channels = lead - 1
            else:
                add_channels = 0



        training_input = np.empty(shape = (training_size_shape[0],training_size_shape[1],training_size_shape[2],len(lag_integer_list)+len(var_list)*3+add_channels))
        validation_input = np.empty(shape = (validation_testing_size_shape[0],validation_testing_size_shape[1],validation_testing_size_shape[2],len(lag_integer_list)+len(var_list)*3+add_channels))
        testing_input = np.empty(shape = (validation_testing_size_shape[0],validation_testing_size_shape[1],validation_testing_size_shape[2],len(lag_integer_list)+len(var_list)*3+add_channels))

        print(f'Training shape input is {training_input.shape}')

        #Add RZSM observations first
        for idx,lag in enumerate(lag_integer_list):
            channel_list.append(f'RZSM_obs_lag{lag}')
            training_input, validation_input, testing_input = pred.add_obs_RZSM_by_lag(training_input, validation_input, testing_input, lag, input_directory, idx, final_testing_year)

        print(f'Index idx value is {idx}. Done adding RZSM obs.')
        
        RZSM_train_obs, RZSM_validation_obs = pred.return_masking_objects_for_RZSM(input_directory,final_testing_year)

        
        for variable in var_list:
            channel_name_output = pred.return_channel_name(variable)
            for lag in observation_lag_list_not_RZSM:
                channel_list.append(f'{channel_name_output}_obs_lag{lag}')
                idx+=1
                #Observations adding
                training_input, validation_input, testing_input = pred.add_obs_other_observations_by_lag(training_input, validation_input, testing_input, lag, input_directory, variable, idx, final_testing_year)
     
        
            print(f'Index idx value is {idx}. Done adding {variable} obs.')
    
        if lead in [0,1]:
            pass
        else:
            #Now we need to load our data from the previous weeks
            for lead_previous in range(1,lead):
                print(f'Working on previous lead: Week {lead_previous}')
                # break
    
                pred_dir = f'predictions/{region_name}/Wk{lead_previous}_training_validation'
                testing_dir = f'predictions/{region_name}/Wk{lead_previous}_testing'
    
                os.system(f'mkdir -p {pred_dir}')
    
                training_prediction_name= f'{pred_dir}/Wk{lead_previous}_training_{experiment_name_out}.npy'
                validation_prediction_name = f'{pred_dir}/Wk{lead_previous}_validation_{experiment_name_out}.npy'
                testing_prediction_name = f'{testing_dir}/Wk{lead_previous}_testing_{experiment_name_out}.npy'
    
                testing_prediction = np.load(testing_prediction_name)
                
                '''Load the training and validation predictions if they exist'''
                if os.path.exists(training_prediction_name):
                    print(f'Loading data from previous week lead {lead_previous}')
                    training_prediction = np.load(training_prediction_name)
                    validation_prediction =np.load(validation_prediction_name)
    
                else:
                    print(f'Creating prediction data from previous week lead {lead_previous}')
                    '''If they don't exist..... load model for previous week'''
                    model = load_model(f'checkpoints/Wk{lead_previous}/Wk{lead_previous}_{experiment_name_out}',compile=False) #don't need the custom loss function for predictions
    
                    training_input_previous = np.load(f'Data/model_npy_inputs/{region_name}/Wk{lead_previous}_EX_input_data/{experiment_name_out}_training_input.npy')
                    validation_input_previous = np.load(f'Data/model_npy_inputs/{region_name}/Wk{lead_previous}_EX_input_data/{experiment_name_out}_validation_input.npy')
                    testing_input_previous = np.load(f'Data/model_npy_inputs/{region_name}/Wk{lead_previous}_EX_input_data/{experiment_name_out}_testing_input.npy')
    
                    '''Make predictions and save data to load for previous use'''
                    training_prediction = model.predict(training_input_previous) #Just for unit checking
                    np.save(training_prediction_name,training_prediction)
    
                    validation_prediction = model.predict(validation_input_previous)
                    np.save(validation_prediction_name,validation_prediction)
    
                ############# ADD BACK TO CREATED ARRAY ###############################
                '''Now get the best UNET index prediction for RZSM and Tmax. Because the UNET plus plus produces 4 different outputs'''
                min_RZSM_index,min_tmax_index = f.load_loss_csv_bias_correction(lead_previous,experiment_name_out,RZSM_or_Tmax_or_both,region_name)
    
                if np.array(training_prediction).shape[0] == 8 and RZSM_or_Tmax_or_both == 'both':
                    min_tmax_index = min_tmax_index+4 #Because index values 0-3 are RZSM and 4-7 are Tmax
                elif np.array(training_prediction).shape[0] == 6 and RZSM_or_Tmax_or_both == 'both':
                    min_tmax_index = min_tmax_index + 3 #Because index values are 0-2 for RZSM and 3-5 for Tmax
                '''Now split the training and validation data back into RZSM and/or Tmax'''
    
    
                train_RZSM = np.array(training_prediction)[min_RZSM_index,:,:,:,:]
                val_RZSM = np.array(validation_prediction)[min_RZSM_index,:,:,:,:]
                test_RZSM = np.array(testing_prediction)[min_RZSM_index,:,:,:,:]

    
                '''Make sure we mask RZSM values which aren't on land'''
                train_RZSM = np.where(RZSM_train_obs==0,0,train_RZSM)
                val_RZSM = np.where(RZSM_validation_obs==0,0,val_RZSM)
                test_RZSM = np.where(RZSM_validation_obs==0,0,test_RZSM)
    
                #Now add back to the data
                '''Now add back to the newly created dataset'''
                idx+=1
                channel_list.append(f'RZSM_prediction_lead{lead_previous}')
                print(f'Adding RZSM training, validation, testing into index {idx}')
                training_input[:,:,:,idx] = train_RZSM[:,:,:,0]
                validation_input[:,:,:,idx] = val_RZSM[:,:,:,0]
                testing_input[:,:,:,idx] = test_RZSM[:,:,:,0]
    
            if RZSM_or_Tmax_or_both == 'both':
                channel_list.append(f'tmax_prediction_lead{lead_previous}')
                idx+=1
                print(f'Adding Tmax training, validation, testing into index {idx}')
                training_input[:,:,:,idx] = train_tmax[:,:,:,0]
                validation_input[:,:,:,idx] = val_tmax[:,:,:,0]
                testing_input[:,:,:,idx] = test_tmax[:,:,:,0]
    

            print('Final channel of input is the following (just make sure it has zeros for RZSM)')
            print(training_input[0,:,:,-1])

    elif addtl_experiment == True:
        #Get the number of extra channels to add from previous weeks predictions
        add_channels = 0


        training_input = np.empty(shape = (training_size_shape[0],training_size_shape[1],training_size_shape[2],len(lag_integer_list)+len(observation_lag_list_not_RZSM)*5+add_channels))
        validation_input = np.empty(shape = (validation_testing_size_shape[0],validation_testing_size_shape[1],validation_testing_size_shape[2],len(lag_integer_list)+len(observation_lag_list_not_RZSM)*5+add_channels))
        testing_input = np.empty(shape = (validation_testing_size_shape[0],validation_testing_size_shape[1],validation_testing_size_shape[2],len(lag_integer_list)+len(observation_lag_list_not_RZSM)*5+add_channels))


        #Add RZSM observations first
        for idx,lag in enumerate(lag_integer_list):
            channel_list.append(f'RZSM_obs_lag{lag}')
            training_input, validation_input, testing_input = pred.add_obs_RZSM_by_lag(training_input, validation_input, testing_input, lag, input_directory, idx, final_testing_year)

        print(f'Index idx value is {idx}. Done adding RZSM obs.')    
        
        RZSM_train_obs, RZSM_validation_obs = pred.return_masking_objects_for_RZSM(input_directory, final_testing_year)


        for variable in var_list:
            channel_name_output = pred.return_channel_name(variable)
            for lag in observation_lag_list_not_RZSM:
                channel_list.append(f'{channel_name_output}_obs_lag{lag}')
                idx+=1
                #Observations adding
                training_input, validation_input, testing_input = pred.add_obs_other_observations_by_lag(training_input, validation_input, testing_input, lag, input_directory, variable, idx,final_testing_year)
     
        
            print(f'Index idx value is {idx}. Done adding {variable} obs.')

        print('Final channel of input is the following (just make sure it has zeros for RZSM)')
        print(training_input[0,:,:,-1])

    return(training_input, validation_input, testing_input,channel_list)

In [6]:
def load_all_data_EX5_EX6_EX7_EX8_experiments(lead, num_lags_obs_RZSM, include_lags_obs_pwat_spfh_tmax, 
                                              include_reforecast_or_not, observation_lag_list_not_RZSM, lag_integer_list,
                                              input_directory,training_size_shape,validation_testing_size_shape,RZSM_or_Tmax_or_both,experiment_test,experiment_name,experiment_name_out):
    print('\nUsing observations as predictions and forecast lead week 1\n')
    #Observations RZSM
    '''We need to combine all the RZSM files into 1 single dictionary for later processing'''
    
    channel_list = []

    assert lead >=1, 'We are only looking at weekly leads 1-5, cannot do 0 lead'
    
    #Reforecast predictors
    var_list = ['pwat_eatm', 'spfh_2m', 'tmax_2m', 'diff_temp_2m', 'hgt_pres']
    
    print(f'Experiment name is {experiment_name_out}')
    
    if experiment_name in ['EX22','EX23','EX24','EX25']:
        #This is only a bias correction using observations (no reforecast input at all)
        lead_add = 0
    elif experiment_name in ['EX5','EX6','EX7','EX8']:
        #This is only a bias correction using observations and the current week of reforecast as inputs
        lead_add = 1
    elif experiment_name in ['EX18','EX19','EX20','EX21']:
        if lead ==1:
            lead_add = 1 #Only adding the current week of forecast
        else:
            lead_add = lead #Adding previous weeks forecast and the current week

    print(f'Adding {lead_add} to channel list from reforecast.')

        
    training_input = np.empty(shape = (training_size_shape[0],training_size_shape[1],training_size_shape[2],len(lag_integer_list)+ lead_add))
    validation_input = np.empty(shape = (validation_testing_size_shape[0],validation_testing_size_shape[1],validation_testing_size_shape[2],len(lag_integer_list) + lead_add))
    testing_input = np.empty(shape = (validation_testing_size_shape[0],validation_testing_size_shape[1],validation_testing_size_shape[2],len(lag_integer_list)+ lead_add))
    
    print(f'Training shape: {training_input.shape}')
    
    #Add RZSM observations first
    for idx,lag in enumerate(lag_integer_list):
        channel_list.append(f'RZSM_obs_lag{lag}')
        training_input, validation_input, testing_input = pred.add_obs_RZSM_by_lag(training_input, validation_input, testing_input, lag, input_directory, idx,final_testing_year)

    print(f'Index idx value is {idx}. Done adding RZSM obs.')
    
    #For masking
    RZSM_train_obs, RZSM_validation_obs = pred.return_masking_objects_for_RZSM(input_directory,final_testing_year)

    if experiment_name in ['EX5','EX6','EX7','EX8']:
        #We only need to add the current reforecast week
        #Add final predictor for the current week of RZSM within reforecast
        idx+=1
        soil_var = 'soilw_bgrnd'
        channel_name_output = pred.return_channel_name(soil_var)
        channel_list.append(f'{channel_name_output}_ref_lead{lead}')
    
        print(f'And finally adding RZSM week {lead} from reforecast as a predictor')
        
        training_input, validation_input, testing_input = pred.add_reforecast_by_lag(training_input, validation_input, testing_input, lead, input_directory,soil_var, idx, ref_source, final_testing_year)
        
        return(training_input, validation_input, testing_input, channel_list)

    elif experiment_name in ['EX22','EX23','EX24','EX25']:
         #We are not adding any additional predictors
         return(training_input, validation_input, testing_input, channel_list)
        
    
    else:
        assert lead >= 2, 'If trying to run EX18-EX21, DO NOT RUN with week 1 data. Start with Week 2'

        #Now we need to load our data from the previous weeks
        for lead_previous in range(1,lead):

            #To not have to re-run a bunch more models, we can re-use the training from a previous week

            if lead_previous == 1:
                if experiment_name == 'EX18':
                    prev_experiment = f'EX5_regular_RZSM'
                        
                elif experiment_name == 'EX19':
                    prev_experiment = f'EX6_regular_RZSM'    
                    
                elif experiment_name == 'EX20':
                    prev_experiment = f'EX7_regular_RZSM'  
                    
                elif experiment_name == 'EX21':
                    prev_experiment = f'EX8_regular_RZSM'
            else:
                prev_experiment = experiment_name_out


            print(f'Working on previous lead: Week {lead_previous}')
            # break

            pred_dir = f'predictions/{region_name}/Wk{lead_previous}_training_validation'
            testing_dir = f'predictions/{region_name}/Wk{lead_previous}_testing'

            os.system(f'mkdir -p {pred_dir}')

            training_prediction_name= f'{pred_dir}/Wk{lead_previous}_training_{prev_experiment}.npy'
            validation_prediction_name = f'{pred_dir}/Wk{lead_previous}_validation_{prev_experiment}.npy'
            testing_prediction_name = f'{testing_dir}/Wk{lead_previous}_testing_{prev_experiment}.npy'

    
            testing_prediction = np.load(testing_prediction_name)
            '''Load the training and validation predictions if they exist'''
            
            if os.path.exists(training_prediction_name):
                print(f'Loading data from previous week lead {lead_previous}')
                training_prediction = np.load(training_prediction_name)
                validation_prediction =np.load(validation_prediction_name)

            else:
                print(f'Creating prediction data from previous week lead {lead_previous} from {prev_experiment}.')
                '''If they don't exist..... load model for previous week'''
                model = load_model(f'checkpoints/{region_name}/Wk{lead_previous}/Wk{lead_previous}_{prev_experiment}',compile=False) #don't need the custom loss function for predictions

                training_input_previous = np.load(f'Data/model_npy_inputs/{region_name}/Wk{lead_previous}_EX_input_data/{prev_experiment}_training_input.npy')
                validation_input_previous = np.load(f'Data/model_npy_inputs/{region_name}/Wk{lead_previous}_EX_input_data/{prev_experiment}_validation_input.npy')
                testing_input_previous = np.load(f'Data/model_npy_inputs/{region_name}/Wk{lead_previous}_EX_input_data/{prev_experiment}_testing_input.npy')

                '''Make predictions and save data to load for previous use'''
                training_prediction = model.predict(training_input_previous) #Just for unit checking
                np.save(training_prediction_name,training_prediction)

                validation_prediction = model.predict(validation_input_previous)
                np.save(validation_prediction_name,validation_prediction)

            ############# ADD BACK TO CREATED ARRAY ###############################

            train_RZSM = np.array(training_prediction)[-1,:,:,:,:]
            val_RZSM = np.array(validation_prediction)[-1,:,:,:,:]
            test_RZSM = np.array(testing_prediction)[-1,:,:,:,:]


            '''Make sure we mask RZSM values which aren't on land'''
            train_RZSM = np.where(RZSM_train_obs==0,0,train_RZSM)
            val_RZSM = np.where(RZSM_validation_obs==0,0,val_RZSM)
            test_RZSM = np.where(RZSM_validation_obs==0,0,test_RZSM)

            #Now add back to the data
            '''Now add back to the newly created dataset'''
            idx+=1
            channel_list.append(f'RZSM_prediction_lead{lead_previous}')
            print(f'Adding RZSM training, validation, testing into index {idx}')
            training_input[:,:,:,idx] = train_RZSM[:,:,:,0]
            validation_input[:,:,:,idx] = val_RZSM[:,:,:,0]
            testing_input[:,:,:,idx] = test_RZSM[:,:,:,0]


        #Add final predictor for the current week of RZSM within reforecast
        idx+=1
        soil_var = 'soilw_bgrnd'
        channel_name_output = pred.return_channel_name(soil_var)
        channel_list.append(f'{channel_name_output}_ref_lead{lead}')
    
        print(f'And finally adding RZSM week {lead} from reforecast as a predictor')
        
        training_input, validation_input, testing_input = pred.add_reforecast_by_lag(training_input, validation_input, testing_input, lead, input_directory,soil_var, idx, ref_source, final_testing_year)
    
        print(f'Index idx value is {idx}. Done adding {soil_var} reforecast.')

        print('Final channel of input is the following (just make sure it has zeros for RZSM)')
        print(training_input[0,:,:,-1])


        return(training_input, validation_input, testing_input, channel_list)

In [7]:
def load_all_data_EX9_EX10_EX11_EX12_after_week_0(lead, num_lags_obs_RZSM, include_lags_obs_pwat_spfh_tmax, include_reforecast_or_not, 
                                           observation_lag_list_not_RZSM, lag_integer_list, input_directory,training_size_shape,
                                           validation_testing_size_shape,experiment_name,RZSM_or_Tmax_or_both, experiment_test,region_name,experiment_name_out):
    '''We need to combine all the RZSM files and all the observation data (pwat, spfh, tmax, diff_temp, z200) into one file. Also adding the RZSM reforecast data from RZSM and Tmax '''

    channel_list = []
    
    #Reforecast predictors
    var_list = ['pwat_eatm', 'spfh_2m', 'tmax_2m', 'diff_temp_2m', 'hgt_pres']

    if experiment_test == 0:
        add_channels = lead  
        reforecast_predictors = False
    elif (experiment_test >= 1) and (lead in [1,2]):
        #This is adding 5 additional predictors (tmax, diff_temp, z200, pwat, spfh)
        #Also add the number of channels according to the lead. If lead == 1, then only add lead 1 current week. If lead == 2, add one channel for week 2 and one channel for the prediction from Week 1
        add_channels = lead  + len(var_list)
        reforecast_predictors = True
    elif (experiment_test >= 1) and (lead > 2):
        add_channels = lead 
        reforecast_predictors = False
            
   
    training_input = np.empty(shape = (training_size_shape[0],training_size_shape[1],training_size_shape[2],len(lag_integer_list)+len(var_list)*3+add_channels))
    validation_input = np.empty(shape = (validation_testing_size_shape[0],validation_testing_size_shape[1],validation_testing_size_shape[2],len(lag_integer_list)+len(var_list)*3+add_channels))
    testing_input = np.empty(shape = (validation_testing_size_shape[0],validation_testing_size_shape[1],validation_testing_size_shape[2],len(lag_integer_list)+len(var_list)*3+add_channels))
    
    print(f'Training input shape = {training_input.shape}')
    
    #Add RZSM observations first
    for idx,lag in enumerate(lag_integer_list):
        channel_list.append(f'RZSM_obs_lag{lag}')
        training_input, validation_input, testing_input = pred.add_obs_RZSM_by_lag(training_input, validation_input, testing_input, lag, input_directory, idx, final_testing_year)
        
    print(f'Index idx value is {idx}. Done adding RZSM obs.')    
    
    RZSM_train_obs, RZSM_validation_obs = pred.return_masking_objects_for_RZSM(input_directory, final_testing_year)

    for variable in var_list:
        channel_name_output = pred.return_channel_name(variable)
        for lag in observation_lag_list_not_RZSM:
            channel_list.append(f'{channel_name_output}_obs_lag{lag}')
            idx+=1
            #Observations adding
            training_input, validation_input, testing_input = pred.add_obs_other_observations_by_lag(training_input, validation_input, testing_input, lag, input_directory, variable, idx,final_testing_year)
 
    
        print(f'Index idx value is {idx}. Done adding {variable} obs.')
    
    ############################### NOW ADD THE PREDICTION DATA FROM PREVIOUS WEEK ################################################
    #Now we need to load our data from the previous weeks 
    
    if lead in [0, 1]:
        #DO NOT INCLUDE LEAD 0 AS A PREDICTOR (FROM PREVIOUS WEEK)
        pass
    else:

        for lead_previous in range(1,lead):
            print(f'Adding previous RZSM prediction from week {lead_previous} as an input channel')
            # break
            
            pred_dir = f'predictions/{region_name}/Wk{lead_previous}_training_validation'
            testing_dir = f'predictions/{region_name}/Wk{lead_previous}_testing'
            
            os.system(f'mkdir -p {pred_dir}')
            
            training_prediction_name= f'{pred_dir}/Wk{lead_previous}_training_{experiment_name_out}.npy'
            validation_prediction_name = f'{pred_dir}/Wk{lead_previous}_validation_{experiment_name_out}.npy'
            testing_prediction_name = f'{testing_dir}/Wk{lead_previous}_testing_{experiment_name_out}.npy'
            
            testing_prediction = np.load(testing_prediction_name)
            
            '''Load the training and validation predictions if they exist'''
            if os.path.exists(training_prediction_name):
                print(f'Loading data from previous week lead {lead_previous}')
                training_prediction = np.load(training_prediction_name)
                validation_prediction =np.load(validation_prediction_name)
                
            else:
                print(f'Creating prediction data from previous week lead {lead_previous}')
                '''If they don't exist..... load model for previous week'''
                model = load_model(f'checkpoints/Wk_{lead_previous}/Wk{lead_previous}_{experiment_name_out}',compile=False) #don't need the custom loss function for predictions
                
                training_input_previous = np.load(f'Data/model_npy_inputs/{region_name}/Wk{lead_previous}_EX_input_data/{experiment_name_out}_training_input.npy')
                validation_input_previous = np.load(f'Data/model_npy_inputs/{region_name}/Wk{lead_previous}_EX_input_data/{experiment_name_out}_validation_input.npy')
                testing_input_previous = np.load(f'Data/model_npy_inputs/{region_name}/Wk{lead_previous}_EX_input_data/{experiment_name_out}_testing_input.npy')
                
                '''Make predictions and save data to load for previous use'''
                training_prediction = model.predict(training_input_previous) #Just for unit checking
                np.save(training_prediction_name,training_prediction)
                
                validation_prediction = model.predict(validation_input_previous)
                np.save(validation_prediction_name,validation_prediction)
            
            
            ############# ADD BACK TO CREATED ARRAY ###############################
            '''Now get the best UNET index prediction for RZSM and Tmax. Because the UNET plus plus produces 4 different outputs'''
            min_RZSM_index,min_tmax_index = f.load_loss_csv_bias_correction(lead_previous,experiment_name_out,RZSM_or_Tmax_or_both,region_name)
    
            if np.array(training_prediction).shape[0] == 8 and RZSM_or_Tmax_or_both == 'both':
                min_tmax_index = min_tmax_index+4 #Because index values 0-3 are RZSM and 4-7 are Tmax
            elif np.array(training_prediction).shape[0] == 6 and RZSM_or_Tmax_or_both == 'both':
                min_tmax_index = min_tmax_index + 3 #Because index values are 0-2 for RZSM and 3-5 for Tmax
            '''Now split the training and validation data back into RZSM and/or Tmax'''
            
            
            train_RZSM = np.array(training_prediction)[min_RZSM_index,:,:,:,:]
            val_RZSM = np.array(validation_prediction)[min_RZSM_index,:,:,:,:]
            test_RZSM = np.array(testing_prediction)[min_RZSM_index,:,:,:,:]

    
            '''Make sure we mask RZSM values which aren't on land'''
            train_RZSM = np.where(RZSM_train_obs==0,0,train_RZSM)
            val_RZSM = np.where(RZSM_validation_obs==0,0,val_RZSM)
            test_RZSM = np.where(RZSM_validation_obs==0,0,test_RZSM)
    
            #Now add back to the data
            '''Now add back to the newly created dataset'''
            idx+=1
            channel_list.append(f'RZSM_prediction_lead{lead_previous}')
            print(f'Adding RZSM training, validation, testing into index {idx}')
            training_input[:,:,:,idx] = train_RZSM[:,:,:,0]
            validation_input[:,:,:,idx] = val_RZSM[:,:,:,0]
            testing_input[:,:,:,idx] = test_RZSM[:,:,:,0]

    
    ############################### NOW ADD THE REFORECAST DATA FROM THE WEEK TO BE PREDICTED ################################################
    
    #Add predictors from the reforecast model
    if (lead in [1,2]) and (experiment_test == 1):
        print(f'Adding current reforecast data from week {lead} for vars {(var_list)}')  
        #Only add these for week 1 and 2. 
        for variable in var_list:
            idx+=1
            channel_name_output = pred.return_channel_name(variable)
            
            channel_list.append(f'{channel_name_output}_ref_lead{lead}')
            
            training_input, validation_input, testing_input = pred.add_reforecast_by_lag(training_input, validation_input, testing_input, lead, input_directory, variable, idx, ref_source, final_testing_year)
    
            print(f'Index idx value is {idx}. Done adding {variable} reforecast.')

    #Add final predictor for the current week of RZSM within reforecast
    idx+=1
    soil_var = 'soilw_bgrnd'
    channel_name_output = pred.return_channel_name(soil_var)
    channel_list.append(f'{channel_name_output}_ref_lead{lead}')

    print(f'And finally adding RZSM week {lead} from reforecast as a predictor')
    
    training_input, validation_input, testing_input = pred.add_reforecast_by_lag(training_input, validation_input, testing_input, lead, input_directory,soil_var, idx, ref_source, final_testing_year)

    print(f'Index idx value is {idx}. Done adding {soil_var} reforecast.')

    print('Final channel of input is the following (just make sure it has zeros for RZSM)')
    print(training_input[0,:,:,-1])

    return(training_input, validation_input, testing_input,channel_list)




In [8]:
def load_all_data_EX29_after_week_0(lead, num_lags_obs_RZSM, include_lags_obs_pwat_spfh_tmax, include_reforecast_or_not, 
                                           observation_lag_list_not_RZSM, lag_integer_list, input_directory,training_size_shape,
                                           validation_testing_size_shape,experiment_name,RZSM_or_Tmax_or_both, experiment_test,region_name,experiment_name_out):
    '''We need to combine all the RZSM files and all the observation data (pwat, spfh, tmax, diff_temp, z200) into one file. Also adding the RZSM reforecast data from RZSM and Tmax '''

    channel_list = []
    
    #Reforecast predictors
    var_list = ['pwat_eatm', 'spfh_2m', 'tmax_2m', 'diff_temp_2m', 'hgt_pres']

    if (experiment_test == 2) and (lead ==1):
        #This is adding 5 additional predictors (tmax, diff_temp, z200, pwat, spfh)
        #Also add the number of channels according to the lead. If lead == 1, then only add lead 1 current week. If lead == 2, add one channel for week 2 and one channel for the prediction from Week 1
        add_channels = len(lag_integer_list)+len(var_list)*2  #Only add a single prediction week from these
        reforecast_predictors = True
    elif (experiment_test == 2) and (lead ==2):
        add_channels = len(lag_integer_list) + len(var_list)*2 + (lead -1) 
        reforecast_predictors = True   
    elif (experiment_test == 2) and (lead > 2):
        add_channels = len(lag_integer_list) + (lead -1) 
        reforecast_predictors = False  
        
    training_input = np.empty(shape = (training_size_shape[0],training_size_shape[1],training_size_shape[2],add_channels))
    validation_input = np.empty(shape = (validation_testing_size_shape[0],validation_testing_size_shape[1],validation_testing_size_shape[2],add_channels))
    testing_input = np.empty(shape = (validation_testing_size_shape[0],validation_testing_size_shape[1],validation_testing_size_shape[2],add_channels))
    
    print(f'Training input shape = {training_input.shape}')
    
    #Add RZSM observations first
    for idx,lag in enumerate(lag_integer_list):
        channel_list.append(f'RZSM_obs_lag{lag}')
        training_input, validation_input, testing_input = pred.add_obs_RZSM_by_lag(training_input, validation_input, testing_input, lag, input_directory, idx, final_testing_year)
        
    print(f'Index idx value is {idx}. Done adding RZSM obs.')    
    
    RZSM_train_obs, RZSM_validation_obs = pred.return_masking_objects_for_RZSM(input_directory, final_testing_year)

    if lead <=2:
        for variable in var_list:
            channel_name_output = pred.return_channel_name(variable)
            for lag in [-1]:
                '''Only include the first lag'''
                channel_list.append(f'{channel_name_output}_obs_lag{lag}')
                idx+=1
                #Observations adding
                training_input, validation_input, testing_input = pred.add_obs_other_observations_by_lag(training_input, validation_input, testing_input, lag, input_directory, variable, idx,final_testing_year)
     
        
            print(f'Index idx value is {idx}. Done adding {variable} obs.')
        
    ############################### NOW ADD THE PREDICTION DATA FROM PREVIOUS WEEK ################################################
    #Now we need to load our data from the previous weeks 
    
    if lead in [0, 1]:
        #DO NOT INCLUDE LEAD 0 AS A PREDICTOR (FROM PREVIOUS WEEK)
        pass
    else:

        for lead_previous in range(1,lead):
            print(f'Adding previous RZSM prediction from week {lead_previous} as an input channel')
            # break
            
            pred_dir = f'predictions/{region_name}/Wk{lead_previous}_training_validation'
            testing_dir = f'predictions/{region_name}/Wk{lead_previous}_testing'
            
            os.system(f'mkdir -p {pred_dir}')
            
            training_prediction_name= f'{pred_dir}/Wk{lead_previous}_training_{experiment_name_out}.npy'
            validation_prediction_name = f'{pred_dir}/Wk{lead_previous}_validation_{experiment_name_out}.npy'
            testing_prediction_name = f'{testing_dir}/Wk{lead_previous}_testing_{experiment_name_out}.npy'
            
            testing_prediction = np.load(testing_prediction_name)
            
            '''Load the training and validation predictions if they exist'''
            if os.path.exists(training_prediction_name):
                print(f'Loading data from previous week lead {lead_previous}')
                training_prediction = np.load(training_prediction_name)
                validation_prediction =np.load(validation_prediction_name)
                
            else:
                print(f'Creating prediction data from previous week lead {lead_previous}')
                '''If they don't exist..... load model for previous week'''
                model = load_model(f'checkpoints/Wk_{lead_previous}/Wk{lead_previous}_{experiment_name_out}',compile=False) #don't need the custom loss function for predictions
                
                training_input_previous = np.load(f'Data/model_npy_inputs/{region_name}/Wk{lead_previous}_EX_input_data/{experiment_name_out}_training_input.npy')
                validation_input_previous = np.load(f'Data/model_npy_inputs/{region_name}/Wk{lead_previous}_EX_input_data/{experiment_name_out}_validation_input.npy')
                testing_input_previous = np.load(f'Data/model_npy_inputs/{region_name}/Wk{lead_previous}_EX_input_data/{experiment_name_out}_testing_input.npy')
                
                '''Make predictions and save data to load for previous use'''
                training_prediction = model.predict(training_input_previous) #Just for unit checking
                np.save(training_prediction_name,training_prediction)
                
                validation_prediction = model.predict(validation_input_previous)
                np.save(validation_prediction_name,validation_prediction)
            
            
            ############# ADD BACK TO CREATED ARRAY ###############################
            '''Now get the best UNET index prediction for RZSM and Tmax. Because the UNET plus plus produces 4 different outputs'''
            min_RZSM_index,min_tmax_index = f.load_loss_csv_bias_correction(lead_previous,experiment_name_out,RZSM_or_Tmax_or_both,region_name)
    
            if np.array(training_prediction).shape[0] == 8 and RZSM_or_Tmax_or_both == 'both':
                min_tmax_index = min_tmax_index+4 #Because index values 0-3 are RZSM and 4-7 are Tmax
            elif np.array(training_prediction).shape[0] == 6 and RZSM_or_Tmax_or_both == 'both':
                min_tmax_index = min_tmax_index + 3 #Because index values are 0-2 for RZSM and 3-5 for Tmax
            '''Now split the training and validation data back into RZSM and/or Tmax'''
            
            
            train_RZSM = np.array(training_prediction)[min_RZSM_index,:,:,:,:]
            val_RZSM = np.array(validation_prediction)[min_RZSM_index,:,:,:,:]
            test_RZSM = np.array(testing_prediction)[min_RZSM_index,:,:,:,:]

    
            '''Make sure we mask RZSM values which aren't on land'''
            train_RZSM = np.where(RZSM_train_obs==0,0,train_RZSM)
            val_RZSM = np.where(RZSM_validation_obs==0,0,val_RZSM)
            test_RZSM = np.where(RZSM_validation_obs==0,0,test_RZSM)
    
            #Now add back to the data
            '''Now add back to the newly created dataset'''
            idx+=1
            channel_list.append(f'RZSM_prediction_lead{lead_previous}')
            print(f'Adding RZSM training, validation, testing into index {idx}')
            training_input[:,:,:,idx] = train_RZSM[:,:,:,0]
            validation_input[:,:,:,idx] = val_RZSM[:,:,:,0]
            testing_input[:,:,:,idx] = test_RZSM[:,:,:,0]

    
    ############################### NOW ADD THE REFORECAST DATA FROM THE WEEK TO BE PREDICTED ################################################
    
    #Add predictors from the reforecast model
    if (lead in [1,2]):
        print(f'Adding current reforecast data from week {lead} for vars {(var_list)}')  
        #Only add these for week 1 and 2. 
        for variable in var_list:
            idx+=1
            channel_name_output = pred.return_channel_name(variable)
            
            channel_list.append(f'{channel_name_output}_ref_lead{lead}')
            
            training_input, validation_input, testing_input = pred.add_reforecast_by_lag(training_input, validation_input, testing_input, lead, input_directory, variable, idx, ref_source, final_testing_year)
    
            print(f'Index idx value is {idx}. Done adding {variable} reforecast.')

    print(training_input[0,:,:,-1])

    return(training_input, validation_input, testing_input,channel_list)




In [9]:
def load_only_predictions_EX26(lead, input_directory,training_size_shape,validation_testing_size_shape,RZSM_or_Tmax_or_both,experiment_name,region_name,
                               experiment_name_out):
    '''We need to combine all the RZSM files and all the observation data (pwat, spfh, tmax, diff_temp, z200) into one file. Also adding the RZSM reforecast data from RZSM and Tmax '''

    channel_list = []

    assert lead >=2, 'We cannot make a prediction if lead week is 1 because we dont have an input. So make lead week >= 2.'

    #Get the number of extra channels to add from previous weeks predictions
    add_channels = lead # We are only adding the previous weeks lag and the current week. 

    training_input = np.empty(shape = (training_size_shape[0],training_size_shape[1],training_size_shape[2],add_channels))
    validation_input = np.empty(shape = (validation_testing_size_shape[0],validation_testing_size_shape[1],validation_testing_size_shape[2],add_channels))
    testing_input = np.empty(shape = (validation_testing_size_shape[0],validation_testing_size_shape[1],validation_testing_size_shape[2],add_channels))
    
    print(f'Training input shape = {training_input.shape}')
    
    #For masking
    RZSM_train_obs, RZSM_validation_obs = pred.return_masking_objects_for_RZSM(input_directory)

    ############################### NOW ADD THE PREDICTION DATA FROM PREVIOUS WEEK ################################################
        #Now we need to load our data from the previous weeks
    for idx,lead_previous in enumerate(range(1,lead)):
        # break
        
        pred_dir = f'predictions/{region_name}/Wk{lead_previous}_training_validation'
        testing_dir = f'predictions/{region_name}/Wk{lead_previous}_testing'
        
        os.system(f'mkdir -p {pred_dir}')
        
        previous_model = return_fully_autoregressive_EX26(lead)
            
        
        training_prediction_name= f'{pred_dir}/Wk{lead_previous}_training_{previous_model[idx]}.npy'
        validation_prediction_name = f'{pred_dir}/Wk{lead_previous}_validation_{previous_model[idx]}.npy'
        testing_prediction_name = f'{testing_dir}/Wk{lead_previous}_testing_{previous_model[idx]}.npy'
        
        testing_prediction = np.load(testing_prediction_name)
        '''Load the training and validation predictions if they exist'''
        
        if os.path.exists(training_prediction_name):
            print(f'Loading data from previous week lead {lead_previous}')
            training_prediction = np.load(training_prediction_name)
            validation_prediction =np.load(validation_prediction_name)
            
        else:
            print(f'Creating prediction data from previous week lead {lead_previous}')
            '''If they don't exist..... load model for previous week'''
            model = load_model(f'checkpoints/Wk_{lead_previous}/Wk{lead_previous}_{experiment_name}',compile=False) #don't need the custom loss function for predictions
            
            training_input_previous = np.load(f'Data/model_npy_inputs/{region_name}/Wk{lead_previous}_EX_input_data/{experiment_name_out}_training_input.npy')
            validation_input_previous = np.load(f'Data/model_npy_inputs/{region_name}/Wk{lead_previous}_EX_input_data/{experiment_name_out}_validation_input.npy')
            testing_input_previous = np.load(f'Data/model_npy_inputs{region_name}//Wk{lead_previous}_EX_input_data/{experiment_name_out}_testing_input.npy')
            
            '''Make predictions and save data to load for previous use'''
            training_prediction = model.predict(training_input_previous) #Just for unit checking
            np.save(training_prediction_name,training_prediction)
            
            validation_prediction = model.predict(validation_input_previous)
            np.save(validation_prediction_name,validation_prediction)
        
        
        ############# ADD BACK TO CREATED ARRAY ###############################
        '''Now get the best UNET index prediction for RZSM and Tmax. Because the UNET plus plus produces 4 different outputs'''
        min_RZSM_index,min_tmax_index = f.load_loss_csv_bias_correction(lead_previous,previous_model[idx],RZSM_or_Tmax_or_both, region_name)

        if np.array(training_prediction).shape[0] == 8 and RZSM_or_Tmax_or_both == 'both':
            min_tmax_index = min_tmax_index+4 #Because index values 0-3 are RZSM and 4-7 are Tmax
        elif np.array(training_prediction).shape[0] == 6 and RZSM_or_Tmax_or_both == 'both':
            min_tmax_index = min_tmax_index + 3 #Because index values are 0-2 for RZSM and 3-5 for Tmax
        '''Now split the training and validation data back into RZSM and/or Tmax'''
        
        
        train_RZSM = np.array(training_prediction)[min_RZSM_index,:,:,:,:]
        val_RZSM = np.array(validation_prediction)[min_RZSM_index,:,:,:,:]
        test_RZSM = np.array(testing_prediction)[min_RZSM_index,:,:,:,:]


        '''Make sure we mask RZSM values which aren't on land'''
        train_RZSM = np.where(RZSM_train_obs==0,0,train_RZSM)
        val_RZSM = np.where(RZSM_validation_obs==0,0,val_RZSM)
        test_RZSM = np.where(RZSM_validation_obs==0,0,test_RZSM)

        #Now add back to the data
        '''Now add back to the newly created dataset'''
        channel_list.append(f'RZSM_prediction_lead{lead_previous}')
        print(f'Adding RZSM training, validation, testing into index {idx}')
        training_input[:,:,:,idx] = train_RZSM[:,:,:,0]
        validation_input[:,:,:,idx] = val_RZSM[:,:,:,0]
        testing_input[:,:,:,idx] = test_RZSM[:,:,:,0]

        idx+=1

    
    ############################### NOW ADD THE REFORECAST DATA FROM THE WEEK TO BE PREDICTED ################################################
    
    #Add RZSM reforecast week N
    soil_var = 'soilw_bgrnd'
    channel_name_output = pred.return_channel_name(soil_var)
    channel_list.append(f'{channel_name_output}_ref_lead{lead}')

    print(f'And finally adding RZSM week {lead} from reforecast as a predictor')
    
    training_input, validation_input, testing_input = pred.add_reforecast_by_lag(training_input, validation_input, testing_input, lead, input_directory,soil_var, idx)

    print(f'Index idx value is {idx}. Done adding {soil_var} reforecast.')

    print('Final channel of input is the following (just make sure it has zeros for RZSM)')
    print(training_input[0,:,:,-1])

    return(training_input, validation_input, testing_input,channel_list)




In [10]:
def make_UNET_stacked_inputs(lead, num_lags_obs_RZSM, include_lags_obs_pwat_spfh_tmax, include_reforecast_or_not, 
                             observation_lag_list_not_RZSM, lag_integer_list, input_directory,training_size_shape,
                             validation_testing_size_shape,experiment_name,RZSM_or_Tmax_or_both,addtl_experiment,experiment_test,region_name,experiment_name_out):
    
    
    #Set restrictions. For EX0, we only want to bias correct the forecasts with week lead 1 RZSM and tmax
    if experiment_name == 'EX0' or experiment_name == 'EX13':
        
        #Step 1 load data 
        print(f'\nWorking on setting up data for Experiment {experiment_name} for lead {lead}\n')
        
        final_input_train, final_input_validation, final_input_testing,channel_list =  loadData.load_all_data_EX0(lead, num_lags_obs_RZSM, include_lags_obs_pwat_spfh_tmax, include_reforecast_or_not, 
                                                                                                     observation_lag_list_not_RZSM, lag_integer_list, input_directory,training_size_shape,
                                                                                                     validation_testing_size_shape,RZSM_or_Tmax_or_both,addtl_experiment,experiment_test,ref_source,final_testing_year)

        print(f'\nInput channels will be only week 1 Tmax and RZSM through the current lead {lead} which are also going to be predicted. There are no observations in this experiment\n')
        
        print('Done')
        
        
    
    ############################################### EXPERIMENTS 1-4 ####################################################################################
    elif experiment_name in ['EX1','EX2','EX3','EX4','EX14','EX15','EX16','EX17']:

        print(f'\nWorking on setting up data for Experiment {experiment_name} for lead {lead}\n')
        
        if lead == 0:
            #Step 1 load data 
            final_input_train, final_input_validation, final_input_testing,channel_list = loadData.load_all_data_EX1_EX2_EX3_EX4(lead, num_lags_obs_RZSM, include_lags_obs_pwat_spfh_tmax, 
                                                                                                                    include_reforecast_or_not, observation_lag_list_not_RZSM, lag_integer_list, input_directory,
                                                                                                                   training_size_shape,validation_testing_size_shape,RZSM_or_Tmax_or_both,
                                                                                                                                   region_name=region_name,ref_source=ref_source,final_testing_year=final_testing_year)
        else:
            final_input_train, final_input_validation, final_input_testing,channel_list = load_all_data_EX1_EX2_EX3_EX4_after_week_0(lead, num_lags_obs_RZSM, include_lags_obs_pwat_spfh_tmax, include_reforecast_or_not, 
                                                                                                                                       observation_lag_list_not_RZSM, lag_integer_list, input_directory,training_size_shape,
                                                                                                                                       validation_testing_size_shape,experiment_name,RZSM_or_Tmax_or_both,addtl_experiment,experiment_test,
                                                                                                                                    experiment_name_out=experiment_name_out,ref_source=ref_source,final_testing_year=final_testing_year)

        
                
        print('Done')
        
        
    
    ############################################### EXPERIMENTS 5-8 ####################################################################################
    elif experiment_name in ['EX5','EX6','EX7','EX8','EX18','EX19','EX20','EX21','EX22','EX23','EX24','EX25']:

        print(f'\nWorking on setting up data for Experiment {experiment_name} for lead {lead}\n')
        
        # #Step 1 load data 
        # if addtl_experiment == False:
        #     final_input_train, final_input_validation, final_input_testing,channel_list = loadData.load_all_data_EX5_EX6_EX7_EX8(lead, num_lags_obs_RZSM, include_lags_obs_pwat_spfh_tmax, 
        #                                                                                                    include_reforecast_or_not, observation_lag_list_not_RZSM, lag_integer_list,
        #                                                                                                    input_directory,training_size_shape,validation_testing_size_shape,RZSM_or_Tmax_or_both,
                                                                                                                                   # region_name=region_name)
        # else:
        final_input_train, final_input_validation, final_input_testing,channel_list = load_all_data_EX5_EX6_EX7_EX8_experiments(lead, num_lags_obs_RZSM, include_lags_obs_pwat_spfh_tmax, include_reforecast_or_not, 
                                                                                      observation_lag_list_not_RZSM, lag_integer_list, input_directory,training_size_shape,
                                                                                      validation_testing_size_shape,RZSM_or_Tmax_or_both,experiment_test,experiment_name,experiment_name_out=experiment_name_out)

        print('Done')
        
        
    
    ############################################### EXPERIMENTS 9-12 ####################################################################################
    elif (experiment_name == 'EX9') or (experiment_name == 'EX10') or (experiment_name == 'EX11') or (experiment_name == 'EX12') or (experiment_name == 'EX27') or (experiment_name == 'EX28'):
        
        print(f'\nWorking on setting up data for Experiment {experiment_name} for lead {lead}\n')

        if lead == 0:
            final_input_train, final_input_validation, final_input_testing,channel_list = loadData.load_all_data_EX9_EX10_EX11_EX12(lead, num_lags_obs_RZSM, include_lags_obs_pwat_spfh_tmax, 
                                                                                                              include_reforecast_or_not, observation_lag_list_not_RZSM, lag_integer_list, 
                                                                                                              input_directory,training_size_shape,validation_testing_size_shape,RZSM_or_Tmax_or_both, 
                                                                                                                                    experiment_test = experiment_test,
                                                                                                                                   region_name=region_name)
        else:
            final_input_train, final_input_validation, final_input_testing,channel_list = load_all_data_EX9_EX10_EX11_EX12_after_week_0(lead, num_lags_obs_RZSM, include_lags_obs_pwat_spfh_tmax, include_reforecast_or_not, 
                                                                                                       observation_lag_list_not_RZSM, lag_integer_list, input_directory,training_size_shape,
                                                                                                   validation_testing_size_shape,experiment_name,RZSM_or_Tmax_or_both,
                                                                                                                                       experiment_test = experiment_test,
                                                                                                                                       region_name = region_name,experiment_name_out=experiment_name_out)
    elif (experiment_name == 'EX26'):

        print(f'\nWorking on setting up data for Experiment {experiment_name} for lead {lead}\n')

        final_input_train, final_input_validation, final_input_testing,
        channel_list = load_only_predictions_EX26(lead, input_directory,training_size_shape,validation_testing_size_shape,RZSM_or_Tmax_or_both,experiment_name,region_name,
                               experiment_name_out)
        print('Done')
        
    ############################################### EXPERIMENTS 29 ####################################################################################
    elif (experiment_name == 'EX29'):
        
        print(f'\nWorking on setting up data for Experiment {experiment_name} for lead {lead}\n')

        final_input_train, final_input_validation, final_input_testing,channel_list = load_all_data_EX29_after_week_0(lead, num_lags_obs_RZSM, include_lags_obs_pwat_spfh_tmax, include_reforecast_or_not, 
                                                                                                   observation_lag_list_not_RZSM, lag_integer_list, input_directory,training_size_shape,
                                                                                               validation_testing_size_shape,experiment_name,RZSM_or_Tmax_or_both,experiment_test = experiment_test,
                                                                                                                                       region_name = region_name,experiment_name_out=experiment_name_out)
                                                                                                        

    return (final_input_train,final_input_validation,final_input_testing,channel_list)


In [11]:
#To try and save on memory, only load certain files at a time
def return_only_train_validation(lead, num_lags_obs_RZSM, include_lags_obs_pwat_spfh_tmax, include_reforecast_or_not, 
                                 observation_lag_list_not_RZSM, lag_integer_list, input_directory,training_size_shape,
                                 validation_testing_size_shape,experiment_name):
    
    reforecast_train_input, reforecast_validation_input, reforecast_testing_input,channel_list \
    = make_UNET_stacked_inputs(lead, num_lags_obs_RZSM, include_lags_obs_pwat_spfh_tmax, 
                               include_reforecast_or_not, observation_lag_list_not_RZSM, 
                               lag_integer_list, input_directory,training_size_shape,validation_testing_size_shape,experiment_name,region_name)

    return(reforecast_train_input, reforecast_validation_input,reforecast_testing_input,channel_list)





def return_testing_data_model_training(lead, num_lags_obs_RZSM, include_lags_obs_pwat_spfh_tmax, 
                               include_reforecast_or_not, observation_lag_list_not_RZSM, 
                               lag_integer_list, input_directory,training_size_shape,validation_testing_size_shape,experiment_name):

    reforecast_train_input, reforecast_validation_input, reforecast_testing_input,channel_list = make_UNET_stacked_inputs(lead, num_lags_obs_RZSM, include_lags_obs_pwat_spfh_tmax, 
                               include_reforecast_or_not, observation_lag_list_not_RZSM, 
                               lag_integer_list, input_directory,training_size_shape,validation_testing_size_shape,experiment_name,region_name)
    return(reforecast_testing_input,channel_list)

# Load Data files

In [12]:
# ## Testing
# region_name=region_name
# lead = lead_week
# num_lags_obs_RZSM=3
# include_lags_obs_pwat_spfh_tmax=True
# include_reforecast_or_not=True
# addtl_experiment = False
# experiment_test = 1

In [13]:
def return_experiment_name_and_lags_and_channel_dir(include_reforecast_or_not, num_lags_obs_RZSM, include_lags_obs_pwat_spfh_tmax, addtl_experiment, experiment_test, region_name):
    
    experiment_name = CE.return_experiment_name(include_reforecast_or_not,num_lags_obs_RZSM,include_lags_obs_pwat_spfh_tmax,addtl_experiment,experiment_test)

    save_experiment_name = f'{experiment_name}_RZSM'
    if testing_scenario == 'None':
        experiment_name_out = f'{experiment_name}_RZSM'
    else:
        experiment_name_out = f'{experiment_name}_{save_loss_name}_RZSM' #Need to manually change this if using different loss functions to keep track of things
    
    global lag_integer_list
    lag_integer_list =  CE.return_num_day_lags_from_weekly_lags(num_lags_obs_RZSM) #For number of RZSM observation lags

    #Where to save channel information
    save_experiment_dir = f'Data/model_npy_inputs/{region_name}/Wk{lead}_EX_input_data'
    
    channel_save_dir = f'channel_list_information/Wk{lead}'
    
    checkpoint_filepath = f'checkpoints/{region_name}/Wk{lead}/Wk{lead}_{experiment_name_out}'
    
    losses_dir = f'Losses_with_OBS/{region_name}/Wk{lead}'
    
    save_checkpoint_dir =f'checkpoints/{region_name}/Wk{lead}'
    
    os.system(f'mkdir -p {checkpoint_filepath} {channel_save_dir} {save_experiment_dir} {losses_dir} {save_checkpoint_dir}')

    
    #Set up files for either saving or loading
    training_input_file = f'{save_experiment_dir}/{save_experiment_name}_training_input.npy'
    validation_input_file = f'{save_experiment_dir}/{save_experiment_name}_validation_input.npy'
    testing_input_file = f'{save_experiment_dir}/{save_experiment_name}_testing_input.npy'

    
    return(save_experiment_name,experiment_name_out,experiment_name,lag_integer_list,channel_save_dir,
           training_input_file,validation_input_file,testing_input_file,checkpoint_filepath,
          losses_dir,save_checkpoint_dir,save_experiment_dir)

In [14]:
def generate_model_inputs(region_name,lead,include_reforecast_or_not,num_lags_obs_RZSM,include_lags_obs_pwat_spfh_tmax,addtl_experiment,experiment_test):

    save_experiment_name, experiment_name_out, experiment_name,lag_integer_list, \
    channel_save_dir, training_input_file, validation_input_file, \
    testing_input_file,checkpoint_filepath, losses_dir,save_checkpoint_dir, \
    save_experiment_dir = return_experiment_name_and_lags_and_channel_dir(include_reforecast_or_not, num_lags_obs_RZSM, include_lags_obs_pwat_spfh_tmax, addtl_experiment, experiment_test, region_name)

    if os.path.exists('redoooooo.nc'):
    # if os.path.exists(training_input_file) and os.path.exists(validation_input_file) and os.path.exists(testing_input_file):
        pass
    else:
        reforecast_train_input, reforecast_validation_input, reforecast_testing_input, channel_list \
        = make_UNET_stacked_inputs(lead, num_lags_obs_RZSM, include_lags_obs_pwat_spfh_tmax, include_reforecast_or_not, 
                             observation_lag_list_not_RZSM, lag_integer_list, input_directory,training_size_shape,
                             validation_testing_size_shape,experiment_name,RZSM_or_Tmax_or_both,addtl_experiment,experiment_test,region_name,experiment_name_out)
    
        #Save data to file
        np.save(training_input_file,tf.convert_to_tensor(reforecast_train_input,dtype=tf.float32))
        np.save(validation_input_file,tf.convert_to_tensor(reforecast_validation_input,dtype=tf.float32))
        np.save(testing_input_file,tf.convert_to_tensor(reforecast_testing_input,dtype=tf.float32))
        
        image_size = reforecast_train_input.shape[1:]           
        #Save channel list information to txt file
    
        with open(f'{channel_save_dir}/{save_experiment_name}_channel_list.txt', 'w') as file:
            for idx,element in enumerate(channel_list):
                if '-' in element:
                    source_='OBSERVATIONS'
                else:
                    source_='REFORECAST'
    
                file.write(f'Channel_{idx} is from {source_} with lead or lag {str(element)}' + '\n')

    return(f'Completed writing lead {lead} model input data to {save_experiment_dir}.')

In [15]:
def train(epochs, batch_size, lead, initial_learning_rate, beta_1,shuffle,patience, kernel_norm,deep_supervision,
         num_lags_obs_RZSM,include_lags_obs_pwat_spfh_tmax,include_reforecast_or_not,
          number_of_UNET_backbone_max_pool,permutation_test,make_additional_predictions_from_model_for_testing,
         input_directory,training_size_shape,validation_testing_size_shape,addtl_experiment,experiment_test,region_name):
    
#For testing
# lead=1
# num_lags_obs_RZSM = 3
# include_lags_obs_pwat_spfh_tmax = True
# include_reforecast_or_not=True
# deep_supervision = True
# initial_learning_rate = 0.0001
# beta_1 = 0.9
# batch_size=66
# epochs=1
# shuffle=False
# kernel_norm =  None
# patience=10
# permutation_test = False
    
    # print(addtl_experiment)
    #Training data
    #Needed for calculating the loss for individual UNET predictions (each has 4 predictions for each variable)
    def return_training_verification_data(RZSM_or_tmax):
        #We had multiple verification inputs in the past, but now we just reduced to RZSM
        if RZSM_or_tmax == 'RZSM':
            return(np.array(tf.expand_dims(obs_final_train,-1)))

    
    #Validation data
    def return_validation_verification_data(RZSM_or_tmax):
        #We had multiple verification inputs in the past, but now we just reduced to RZSM
        if RZSM_or_tmax == 'RZSM':
            return(np.array(tf.expand_dims(obs_final_validation,-1)))

    
    
    def print_shape(file,name):
        print(f'Shape of {name} is {file.shape}')
        
    save_experiment_name, experiment_name_out, experiment_name,lag_integer_list, \
    channel_save_dir, training_input_file, validation_input_file, \
    testing_input_file,checkpoint_filepath, losses_dir,save_checkpoint_dir, \
    save_experiment_dir = return_experiment_name_and_lags_and_channel_dir(include_reforecast_or_not, num_lags_obs_RZSM, include_lags_obs_pwat_spfh_tmax, addtl_experiment, experiment_test, region_name)

    obs_final_train,obs_final_validation,obs_final_testing = f.load_verification_observations_updated(lead,verification_directory)
    

    print('Loading previously created data')
    #Have to do this way because of memory issues
    
    if number_of_UNET_backbone_max_pool == 4:
        if RZSM_or_Tmax_or_both == 'both':
            reforecast_train_input_tensor = tf.data.Dataset.from_tensor_slices(({'input_image':np.load(training_input_file)},
                                                                                {'RZSM_output_1': return_training_verification_data('RZSM'), 'RZSM_output_2': return_training_verification_data('RZSM'), 'RZSM_output_3': return_training_verification_data('RZSM'),
                                                                                 'tmax_output_1': return_training_verification_data('tmax'), 'tmax_output_2': return_training_verification_data('tmax'), 'tmax_output_3': return_training_verification_data('tmax')}))
            reforecast_validation_input_tensor = tf.data.Dataset.from_tensor_slices(({'input_image':np.load(validation_input_file)},
                                                                                {'RZSM_output_1': return_validation_verification_data('RZSM'), 'RZSM_output_2': return_validation_verification_data('RZSM'), 'RZSM_output_3': return_validation_verification_data('RZSM'),
                                                                                 'tmax_output_1': return_validation_verification_data('tmax'), 'tmax_output_2': return_validation_verification_data('tmax'), 'tmax_output_3': return_validation_verification_data('tmax')}))
            # reforecast_testing_input_tensor = tf.data.Dataset.from_tensor_slices((np.load(testing_input_file),obs_final_testing))
            # reforecast_testing_input = np.load(testing_input_file)
        elif RZSM_or_Tmax_or_both == 'RZSM':
            reforecast_train_input_tensor = tf.data.Dataset.from_tensor_slices(({'input_image':np.load(training_input_file)},
                                                                                {'RZSM_output_1': return_training_verification_data('RZSM'), 'RZSM_output_2': return_training_verification_data('RZSM'), 'RZSM_output_3': return_training_verification_data('RZSM')}))
            reforecast_validation_input_tensor = tf.data.Dataset.from_tensor_slices(({'input_image':np.load(validation_input_file)},
                                                                                {'RZSM_output_1': return_validation_verification_data('RZSM'), 'RZSM_output_2': return_validation_verification_data('RZSM'), 'RZSM_output_3': return_validation_verification_data('RZSM')}))

        image_size = np.array(list(reforecast_train_input_tensor.element_spec)[0]['input_image'].shape)
        
    elif number_of_UNET_backbone_max_pool == 5:
        if RZSM_or_Tmax_or_both == 'both':
            reforecast_train_input_tensor = tf.data.Dataset.from_tensor_slices(({'input_image':np.load(training_input_file)},
                                                                                    {'RZSM_output_1': return_training_verification_data('RZSM'), 'RZSM_output_2': return_training_verification_data('RZSM'), 'RZSM_output_3': return_training_verification_data('RZSM'),'RZSM_output_4': return_training_verification_data('RZSM'),
                                                                                     'tmax_output_1': return_training_verification_data('tmax'), 'tmax_output_2': return_training_verification_data('tmax'), 'tmax_output_3': return_training_verification_data('tmax'),'tmax_output_4': return_training_verification_data('tmax')}))
            reforecast_validation_input_tensor = tf.data.Dataset.from_tensor_slices(({'input_image':np.load(validation_input_file)},
                                                                                    {'RZSM_output_1': return_validation_verification_data('RZSM'), 'RZSM_output_2': return_validation_verification_data('RZSM'), 'RZSM_output_3': return_validation_verification_data('RZSM'),'RZSM_output_4': return_validation_verification_data('RZSM'),
                                                                                     'tmax_output_1': return_validation_verification_data('tmax'), 'tmax_output_2': return_validation_verification_data('tmax'), 'tmax_output_3': return_validation_verification_data('tmax'), 'tmax_output_4': return_validation_verification_data('tmax')}))
        elif RZSM_or_Tmax_or_both == 'RZSM':
            reforecast_train_input_tensor = tf.data.Dataset.from_tensor_slices(({'input_image':np.load(training_input_file)},
                                                                                    {'RZSM_output_1': return_training_verification_data('RZSM'), 'RZSM_output_2': return_training_verification_data('RZSM'), 'RZSM_output_3': return_training_verification_data('RZSM'),'RZSM_output_4': return_training_verification_data('RZSM')}))
            reforecast_validation_input_tensor = tf.data.Dataset.from_tensor_slices(({'input_image':np.load(validation_input_file)},
                                                                                    {'RZSM_output_1': return_validation_verification_data('RZSM'), 'RZSM_output_2': return_validation_verification_data('RZSM'), 'RZSM_output_3': return_validation_verification_data('RZSM'),'RZSM_output_4': return_validation_verification_data('RZSM')}))
            # reforecast_testing_input_tensor = tf.data.Dataset.from_tensor_slices((np.load(testing_input_file),obs_final_testing))
            # reforecast_testing_input = np.load(testing_input_file)

        image_size = np.array(list(reforecast_train_input_tensor.element_spec)[0]['input_image'].shape)

        # reforecast_train_input = np.load(training_input_file)
        # reforecast_validation_input  = np.load(validation_input_file)
        # reforecast_testing_input  = np.load(testing_input_file)
        # image_size = np.array(reforecast_train_input).shape
  
    
    with open(f'{channel_save_dir}/{save_experiment_name}_channel_list.txt', 'r') as file:
        # Read all lines from the file into a list
        channel_list = file.readlines()

    # print_shape(reforecast_train_input,'Training input')
    # print_shape(reforecast_validation_input,'Validation input')
    # print_shape(reforecast_testing_input,'Testing input')
    # print_shape(obs_final_train,'Observation verification training')
    # print_shape(obs_final_validation,'Observation verification validation')
    # print_shape(obs_final_testing,'Observation verification testing')
    

    print('Actual channel order list')
    print(channel_list)

    inputs = Input(shape=image_size, name='input_image')
    
    print('\nCompiling model\n')
    if RZSM_or_Tmax_or_both =='both':
        build_model = UNETRzsmTmax.model_build_func(inputs=inputs, output_channels=1, 
                               using_deep_supervision=deep_supervision, 
                               kernel_norm = kernel_norm , var_name='RZSM_Tmax',
                              number_of_UNET_backbone_max_pool = number_of_UNET_backbone_max_pool)
        model = Model(inputs=inputs,
                 outputs = build_model,
                 name="UNET_tmax_RZSM")
        
    elif RZSM_or_Tmax_or_both =='RZSM':
        build_model = UNETRzsm.model_build_func(inputs=inputs, output_channels=1, 
                               using_deep_supervision=deep_supervision, 
                               kernel_norm = kernel_norm , var_name='RZSM',
                              number_of_UNET_backbone_max_pool = number_of_UNET_backbone_max_pool)
        model = Model(inputs=inputs,
                 outputs = build_model,
                 name="UNET_RZSM")


    
    # print(model.summary())

    if number_of_UNET_backbone_max_pool == 4:
        if RZSM_or_Tmax_or_both =='both':
            model.compile(loss= {'tmax_output_1':loss_fn, 'tmax_output_2':loss_fn, 'tmax_output_3':loss_fn, 
                                 'RZSM_output_1':loss_fn, 'RZSM_output_2':loss_fn, 'RZSM_output_3':loss_fn},
                          metrics = 'mae', optimizer = k.optimizers.Adam(learning_rate=initial_learning_rate, beta_1=beta_1),run_eagerly=True)
        elif RZSM_or_Tmax_or_both =='RZSM':
            model.compile(loss= {'RZSM_output_1':loss_fn, 'RZSM_output_2':loss_fn, 'RZSM_output_3':loss_fn},
                      metrics = 'mae', optimizer = k.optimizers.Adam(learning_rate=initial_learning_rate, beta_1=beta_1),run_eagerly=True)
            
    elif number_of_UNET_backbone_max_pool == 5:
        if RZSM_or_Tmax_or_both =='both':
        # With all return channels
            model.compile(loss= {'tmax_output_1':loss_fn, 'tmax_output_2':loss_fn, 'tmax_output_3':loss_fn, 'tmax_output_4':loss_fn,
                                 'RZSM_output_1':loss_fn, 'RZSM_output_2':loss_fn, 'RZSM_output_3':loss_fn, 'RZSM_output_4':loss_fn},
                metrics = 'mae', optimizer = k.optimizers.Adam(learning_rate=initial_learning_rate, beta_1=beta_1),run_eagerly=True)
        elif RZSM_or_Tmax_or_both =='RZSM':
            model.compile(loss= {'RZSM_output_1':loss_fn, 'RZSM_output_2':loss_fn, 'RZSM_output_3':loss_fn, 'RZSM_output_4':loss_fn},
                metrics = 'mae', optimizer = k.optimizers.Adam(learning_rate=initial_learning_rate, beta_1=beta_1),run_eagerly=True)


    '''For some reason I can't save anymore to my home space, so let's just move it to scratch'''

    checkpoint_filepath_out = f'/glade/derecho/scratch/klesinger/{checkpoint_filepath}'
    save_checkpoint_dir_out = f'/glade/derecho/scratch/klesinger/{save_checkpoint_dir}'

    os.system(f'rm {checkpoint_filepath} -r') #must do this so that we can make a soft link
    os.system(f'mkdir -p {save_checkpoint_dir_out} {save_checkpoint_dir}')
    
    model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath_out,
    save_weights_only=True,
    mode='min',
    save_best_only=True)
    
    rlrop = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5)
    
    my_callbacks = [tf.keras.callbacks.EarlyStopping(patience=patience),model_checkpoint_callback, rlrop] 
    
    unet_history=model.fit(
        x=reforecast_train_input_tensor.batch(batch_size), 
        batch_size=batch_size, epochs=epochs, initial_epoch=0, shuffle=shuffle, callbacks=my_callbacks, \
        validation_data=reforecast_validation_input_tensor.batch(batch_size))

    loss_out_df = pd.DataFrame(unet_history.history)
    loss_out_df.to_csv(f'{losses_dir}/Wk{lead}_{experiment_name_out}')


    ########### MODEL CHECKPOINTS ############################

    model.save(f'/glade/work/klesinger/FD_RZSM_deep_learning/{save_checkpoint_dir}/Wk{lead}_{experiment_name_out}')

    #If saving to a scratch location
    # model.save(f'{save_checkpoint_dir_out}/Wk{lead}_{experiment_name_out}')
    # os.system(f'ln -s {save_checkpoint_dir_out}/Wk{lead}_{experiment_name_out} /glade/work/klesinger/FD_RZSM_deep_learning/{save_checkpoint_dir}/Wk{lead}_{experiment_name_out}')
              
    return(0)


    

In [16]:
# # #Testing for model runs
# permutation_test=False
# lead=1
# num_lags_obs_RZSM =3
# include_lags_obs_pwat_spfh_tmax = True
# include_reforecast_or_not=True
# deep_supervision = True
# initial_learning_rate = 0.0001
# beta_1 = 0.9
# batch_size=66
# epochs=1
# shuffle=False
# kernel_norm =  None
# patience=10
# number_of_UNET_backbone_max_pool=4
# make_additional_predictions_from_model_for_testing=False
# addtl_experiment = False

# Run the model for each experiment

In [17]:
'''For experiments, you need to run the script once to make the data. Then restart the kernel to load the data. 
Otherwise the memory leaks are too high'''
def run_EXPERIMENT(lead, num_lags_obs_RZSM,include_lags_obs_pwat_spfh_tmax ,include_reforecast_or_not,addtl_experiment,experiment_test,region_name):
    if lead_week == 5:
        patience=15
    else:
        patience = 10

    #Options for num_lags_obs_RZSM weekly lags [ 3,6,9,12 ] ** this inc
    #Options for num_lags_obs_pwat_spfh_tmax weekly lags [0 weeks or 2 weeks ]

    train(epochs = 300, 
          batch_size = 66, 
          lead=lead_week, 
          initial_learning_rate=0.0001,
          beta_1 = 0.9,
          shuffle=False,
          patience=patience,
          kernel_norm = None,
          deep_supervision = True,
          num_lags_obs_RZSM=num_lags_obs_RZSM, 
          include_lags_obs_pwat_spfh_tmax=include_lags_obs_pwat_spfh_tmax,
          include_reforecast_or_not=include_reforecast_or_not,
          number_of_UNET_backbone_max_pool=4,
          permutation_test = False,
          make_additional_predictions_from_model_for_testing = False,
          input_directory=input_directory,
          training_size_shape=training_size_shape,
          validation_testing_size_shape=validation_testing_size_shape,
          addtl_experiment = addtl_experiment,
          experiment_test =experiment_test,
          region_name = region_name)
    return(0)



In [18]:
def make_single_predictions_testing(lead, include_reforecast_or_not, num_lags_obs_RZSM, include_lags_obs_pwat_spfh_tmax,addtl_experiment,experiment_test, region_name):
    
    save_experiment_name, experiment_name_out, experiment_name,lag_integer_list, \
    channel_save_dir, training_input_file, validation_input_file, \
    testing_input_file,checkpoint_filepath, losses_dir,save_checkpoint_dir, \
    save_experiment_dir = return_experiment_name_and_lags_and_channel_dir(include_reforecast_or_not, num_lags_obs_RZSM, include_lags_obs_pwat_spfh_tmax, addtl_experiment, experiment_test, region_name)

    print(f'Testing input file is: {testing_input_file}')
    ########### MAKE PREDICTIONS ON TRAINING, TESTING, VALIDATION SET ############################
    # model = load_model(f'checkpoints/Wk_{lead}/Wk{lead}_{experiment_name_out}',compile=False)

    print(f'Starting single prediction for Experiment {experiment_name_out}')
    try:
        model = load_model(checkpoint_filepath,compile=False) #don't need the custom loss function for predictions
        
        #Load data
        reforecast_testing_input = np.load(testing_input_file)
        
        ########### SAVE PREDICTIONS (SINGLE PREDICTIONS) ############################
        test_dir = f'predictions/{region_name}/Wk{lead}_testing'
    
        os.system(f'mkdir -p {test_dir}')
    
        mask = masks.load_mask(region_name)
    
        print('\nCurrently only predicting the test dataset\n')
        predictions = model.predict(reforecast_testing_input)
        # predictions.shape
        np.save(f'{test_dir}/Wk{lead}_testing_{experiment_name_out}.npy',predictions)
    except NameError:
        print('Error loading model')
        pass
    except OSError:
        print('Error loading model')
        pass
    
    return(f'Completed Experiment {experiment_name_out}')

In [19]:
def make_single_predictions_training_validation(lead, include_reforecast_or_not, num_lags_obs_RZSM, include_lags_obs_pwat_spfh_tmax,addtl_experiment,experiment_test, region_name):
    
    save_experiment_name, experiment_name_out, experiment_name,lag_integer_list, \
    channel_save_dir, training_input_file, validation_input_file, \
    testing_input_file,checkpoint_filepath, losses_dir,save_checkpoint_dir, \
    save_experiment_dir = return_experiment_name_and_lags_and_channel_dir(include_reforecast_or_not, num_lags_obs_RZSM, include_lags_obs_pwat_spfh_tmax, addtl_experiment, experiment_test, region_name)

    ########### MAKE PREDICTIONS ON TRAINING, TESTING, VALIDATION SET ############################
    # model = load_model(f'checkpoints/Wk_{lead}/Wk{lead}_{experiment_name_out}',compile=False)

    print(f'Starting single prediction for Experiment {experiment_name_out}')
       
    model = load_model(checkpoint_filepath,compile=False) #don't need the custom loss function for predictions
    
    #Load data
    reforecast_training_input = np.load(training_input_file)
    reforecast_validation_input = np.load(validation_input_file)

    
    ########### SAVE PREDICTIONS (SINGLE PREDICTIONS) ############################
    train_val_dir = f'predictions/{region_name}/Wk{lead}_training_validation'

    os.system(f'mkdir -p {train_val_dir}')

    mask = masks.load_mask(region_name)

    print('\nPredicting training and validation input')
    predictions = model.predict(reforecast_training_input)
    # predictions.shape
    np.save(f'{train_val_dir}/Wk{lead}_training_{experiment_name_out}.npy',predictions)
    
    predictions = model.predict(reforecast_validation_input)
    # predictions.shape
    np.save(f'{train_val_dir}/Wk{lead}_validation_{experiment_name_out}.npy',predictions)
    
    return(f'Completed Experiment {experiment_name_out}')

In [20]:
#These are models which only include the reforecast predictions. We are not including any observations into them.

EX0 ={'region_name':region_name, 'num_lags_obs_RZSM': 0, 'include_lags_obs_pwat_spfh_tmax': False, 'include_reforecast_or_not': True, 'addtl_experiment': False, 'experiment_test':0 }
EX13={'region_name':region_name, 'num_lags_obs_RZSM': 0, 'include_lags_obs_pwat_spfh_tmax': False, 'include_reforecast_or_not': True, 'addtl_experiment': True, 'experiment_test':1 }


In [21]:
#Include all predictors for obs but DO NOT include the current reforecast RZSM 

EX1 ={'region_name':region_name, 'num_lags_obs_RZSM': 3, 'include_lags_obs_pwat_spfh_tmax': True, 'include_reforecast_or_not': False, 'addtl_experiment': False, 'experiment_test':0 }
EX2 ={'region_name':region_name, 'num_lags_obs_RZSM': 6, 'include_lags_obs_pwat_spfh_tmax': True, 'include_reforecast_or_not': False, 'addtl_experiment': False, 'experiment_test':0 }
EX3 ={'region_name':region_name, 'num_lags_obs_RZSM': 9, 'include_lags_obs_pwat_spfh_tmax': True, 'include_reforecast_or_not': False, 'addtl_experiment': False, 'experiment_test':0 }
EX4 ={'region_name':region_name, 'num_lags_obs_RZSM': 12, 'include_lags_obs_pwat_spfh_tmax': True, 'include_reforecast_or_not': False, 'addtl_experiment': False, 'experiment_test':0 }


#Include all predictors for obs but DO include the current reforecast RZSM 
EX14 ={'region_name':region_name, 'num_lags_obs_RZSM': 3, 'include_lags_obs_pwat_spfh_tmax': True, 'include_reforecast_or_not': False, 'addtl_experiment': True, 'experiment_test':1 }
EX15 ={'region_name':region_name, 'num_lags_obs_RZSM': 6, 'include_lags_obs_pwat_spfh_tmax': True, 'include_reforecast_or_not': False, 'addtl_experiment': True, 'experiment_test':2 }
EX16 ={'region_name':region_name, 'num_lags_obs_RZSM': 9, 'include_lags_obs_pwat_spfh_tmax': True, 'include_reforecast_or_not': False, 'addtl_experiment': True, 'experiment_test':3 }
EX17 ={'region_name':region_name, 'num_lags_obs_RZSM': 12, 'include_lags_obs_pwat_spfh_tmax': True, 'include_reforecast_or_not': False, 'addtl_experiment': True, 'experiment_test':4 }


In [22]:
#Include only OBS RZSM predictors and the current reforecast RZSM week 

EX5 ={'region_name':region_name, 'num_lags_obs_RZSM': 3, 'include_lags_obs_pwat_spfh_tmax': False, 'include_reforecast_or_not': True, 'addtl_experiment': False, 'experiment_test':0 }
EX6 ={'region_name':region_name, 'num_lags_obs_RZSM': 6, 'include_lags_obs_pwat_spfh_tmax': False, 'include_reforecast_or_not': True, 'addtl_experiment': False, 'experiment_test':0 }
EX7 ={'region_name':region_name, 'num_lags_obs_RZSM': 9, 'include_lags_obs_pwat_spfh_tmax': False, 'include_reforecast_or_not': True, 'addtl_experiment': False, 'experiment_test':0 }
EX8 ={'region_name':region_name, 'num_lags_obs_RZSM': 12, 'include_lags_obs_pwat_spfh_tmax': False, 'include_reforecast_or_not': True, 'addtl_experiment': False, 'experiment_test':0 }

#Include only OBS RZSM predictors, previous weeks prediction, and current reforecast RZSM week 
EX18 ={'region_name':region_name, 'num_lags_obs_RZSM': 3, 'include_lags_obs_pwat_spfh_tmax': False, 'include_reforecast_or_not': True, 'addtl_experiment': True, 'experiment_test':1 }
EX19 ={'region_name':region_name, 'num_lags_obs_RZSM': 6, 'include_lags_obs_pwat_spfh_tmax': False, 'include_reforecast_or_not': True, 'addtl_experiment': True, 'experiment_test':2 }
EX20 ={'region_name':region_name, 'num_lags_obs_RZSM': 9, 'include_lags_obs_pwat_spfh_tmax': False, 'include_reforecast_or_not': True, 'addtl_experiment': True, 'experiment_test':3 }
EX21 ={'region_name':region_name, 'num_lags_obs_RZSM': 12, 'include_lags_obs_pwat_spfh_tmax': False, 'include_reforecast_or_not': True, 'addtl_experiment': True, 'experiment_test':4 }

#Include only OBS RZSM predictors DO NOT INCLUDE ANY REFORECAST OR PREDICTION
EX22 ={'region_name':region_name, 'num_lags_obs_RZSM': 3, 'include_lags_obs_pwat_spfh_tmax': False, 'include_reforecast_or_not': True, 'addtl_experiment': True, 'experiment_test':5 }
EX23 ={'region_name':region_name, 'num_lags_obs_RZSM': 6, 'include_lags_obs_pwat_spfh_tmax': False, 'include_reforecast_or_not': True, 'addtl_experiment': True, 'experiment_test':6 }
EX24 ={'region_name':region_name, 'num_lags_obs_RZSM': 9, 'include_lags_obs_pwat_spfh_tmax': False, 'include_reforecast_or_not': True, 'addtl_experiment': True, 'experiment_test':7 }
EX25 ={'region_name':region_name, 'num_lags_obs_RZSM': 12, 'include_lags_obs_pwat_spfh_tmax': False, 'include_reforecast_or_not': True, 'addtl_experiment': True, 'experiment_test':8 }


In [23]:
#Include only OBS RZSM predictors, OBS other variables, previous predictions of RZSM, and current reforecast lead

EX9 ={'region_name':region_name, 'num_lags_obs_RZSM': 3, 'include_lags_obs_pwat_spfh_tmax': True, 'include_reforecast_or_not': True, 'addtl_experiment': False, 'experiment_test':0 }
EX10 ={'region_name':region_name, 'num_lags_obs_RZSM': 6, 'include_lags_obs_pwat_spfh_tmax': True, 'include_reforecast_or_not': True, 'addtl_experiment': False, 'experiment_test':0 }
EX11 ={'region_name':region_name, 'num_lags_obs_RZSM': 9, 'include_lags_obs_pwat_spfh_tmax': True, 'include_reforecast_or_not': True, 'addtl_experiment': False, 'experiment_test':0 }
EX12 ={'region_name':region_name, 'num_lags_obs_RZSM': 12, 'include_lags_obs_pwat_spfh_tmax': True, 'include_reforecast_or_not': True, 'addtl_experiment': False, 'experiment_test':0 }


In [24]:

# EX26 ={'region_name':region_name, 'num_lags_obs_RZSM': 3, 'include_lags_obs_pwat_spfh_tmax': True, 'include_reforecast_or_not': True, 'addtl_experiment': False, 'experiment_test':0 }

# run_EXPERIMENT(num_lags_obs_RZSM=0,include_lags_obs_pwat_spfh_tmax=False,include_reforecast_or_not=True, addtl_experiment = False, experiment_test = 26)

In [25]:
#This like EX9 and EX10 except now we are adding additional reforecast predictors from week 1 and 2

EX27 ={'region_name':region_name, 'num_lags_obs_RZSM': 3, 'include_lags_obs_pwat_spfh_tmax': True, 'include_reforecast_or_not': True, 'addtl_experiment': False, 'experiment_test':1 }
EX28 ={'region_name':region_name, 'num_lags_obs_RZSM': 6, 'include_lags_obs_pwat_spfh_tmax': True, 'include_reforecast_or_not': True, 'addtl_experiment': False, 'experiment_test':1 }

EX29 ={'region_name':region_name, 'num_lags_obs_RZSM': 3, 'include_lags_obs_pwat_spfh_tmax': True, 'include_reforecast_or_not': True, 'addtl_experiment': False, 'experiment_test':2 }


In [26]:
def run_model_and_prediction(experiment):
    global lead_week
    for lead_week in [1]:
        global lead
        lead=lead_week
    
        generate_model_inputs(region_name=experiment['region_name'], lead = lead_week, num_lags_obs_RZSM=experiment['num_lags_obs_RZSM'],
                              include_lags_obs_pwat_spfh_tmax=experiment['include_lags_obs_pwat_spfh_tmax'],include_reforecast_or_not=experiment['include_reforecast_or_not'], 
                              addtl_experiment = experiment['addtl_experiment'], experiment_test = experiment['experiment_test'])
    
        # run_EXPERIMENT(region_name=experiment['region_name'], lead = lead_week, num_lags_obs_RZSM=experiment['num_lags_obs_RZSM'],
        #                       include_lags_obs_pwat_spfh_tmax=experiment['include_lags_obs_pwat_spfh_tmax'],include_reforecast_or_not=experiment['include_reforecast_or_not'], 
        #                       addtl_experiment = experiment['addtl_experiment'], experiment_test = experiment['experiment_test'])
    
        # make_single_predictions_training_validation(region_name=experiment['region_name'], lead = lead_week, num_lags_obs_RZSM=experiment['num_lags_obs_RZSM'],
        #                       include_lags_obs_pwat_spfh_tmax=experiment['include_lags_obs_pwat_spfh_tmax'],include_reforecast_or_not=experiment['include_reforecast_or_not'], 
        #                       addtl_experiment = experiment['addtl_experiment'], experiment_test = experiment['experiment_test'])

        make_single_predictions_testing(region_name=experiment['region_name'], lead = lead_week, num_lags_obs_RZSM=experiment['num_lags_obs_RZSM'],
                              include_lags_obs_pwat_spfh_tmax=experiment['include_lags_obs_pwat_spfh_tmax'],include_reforecast_or_not=experiment['include_reforecast_or_not'], 
                              addtl_experiment = experiment['addtl_experiment'], experiment_test = experiment['experiment_test'])

        #For permutation test
        # obs_final_train,obs_final_validation,obs_final_testing = verifications.open_obs_for_verification(region_name, [6,13,20,27],train_start, train_end, val_start, val_end, test_start, test_end)
        
    print(f'Completed {experiment}')


In [27]:
all_exps= [EX0,EX1,EX2,EX3,EX4,EX5,EX6,EX7,EX8,EX9,EX10,EX11,EX13,EX14,EX15,EX16,EX17,EX18,EX19,EX20,EX21,EX22,EX23,EX24,EX25,EX27,EX28,EX29]

for experiment in all_exps:
    print(f'Working on experiment {experiment}')
    run_model_and_prediction(experiment = experiment)
t

Working on experiment {'region_name': 'CONUS', 'num_lags_obs_RZSM': 0, 'include_lags_obs_pwat_spfh_tmax': False, 'include_reforecast_or_not': True, 'addtl_experiment': False, 'experiment_test': 0}

Working on setting up data for Experiment EX29 for lead 1

Training input shape = (9185, 48, 96, 13)


mkdir: cannot create directory ‘Data/model_npy_inputs’: File exists


FileNotFoundError: [Errno 2] No such file or directory: 'Data/model_npy_inputs/CONUS/Model_input_data/obs_soilw_bgrnd_GLEAM_training.pickle'

In [ ]:
for ex in [EX1,EX2,EX3,EX4,EX14,EX15,]:
    run_model_and_prediction(experiment = ex)

In [ ]:
stop

# Permutation test only

In [ ]:
def setup_plot_permutation(lead,dictionary,metric):
    test=pd.DataFrame(dictionary,index=[metric])
    return(test)

In [ ]:
def plot_barplot_mae_rmse(save_for_plot_mae, save_for_plot_rmse, lead, model_name):
    save_permutation_figures = f'Outputs/permutation_tests/barplots/{region_name}/Wk{lead}'
    os.system(f'mkdir -p {save_permutation_figures}')
    
    mae = setup_plot_permutation(lead,save_for_plot_mae,'MAE')
    rmse = setup_plot_permutation(lead,save_for_plot_rmse,'RMSE')
       
    # Plotting
    fig, axs = plt.subplots(2)
    
    # Plot for DataFrame 1
    mae.plot(kind='bar', ax=axs[0])
    axs[0].set_title('MAE')
    
    # Plot for DataFrame 2
    rmse.plot(kind='bar', ax=axs[1])
    axs[1].set_title('RMSE')
    
    # Adjust layout
    # plt.tight_layout()
    plt.savefig(f'{save_permutation_figures}/{model_name}.png')
    plt.show()

    return(0)
    

In [ ]:
def create_plot_permutation(lead,max_RZSM_value,region_name):
    save_permutation_figures = f'Outputs/permutation_tests/{region_name}/plots/Wk{lead}'
    os.system(f'mkdir -p {save_permutation_figures}')

    # del plot_['Lead']
    plot_ = plot_.T #Must transpose to make it plot properly

    fig = plt.figure(figsize=(8,6))

    ax = fig.add_subplot(111) # Create matplotlib axes

    plot_.RZSM.plot(kind='bar', color='red',width=0.3, ax=ax, position=1)

    ax.set_ylabel('RZSM anomaly MAE')
    ax.set_ylim(0, max_RZSM_value)

    # plot_.plot(kind='bar')
    # plt.ylabel(f'{error_}')
    plt.suptitle(f'Permutation Test\nObservation values are in legend\nWk{lead} {experiment_name_out}',fontsize=10)
    
    legend1 = ax.legend(loc='upper right', bbox_to_anchor=(1.0, 1))

    ax.set_xlabel('\nPermutated Channels',weight='bold')
    plt.tight_layout()

    plt.savefig(f'{save_permutation_figures}/{experiment_name_out}.png')
    #plt.savefig(f'{save_permutation_figures}/{experiment_name}.tiff', format='tiff', dpi=300)
    #tiff.imsave(f'{save_permutation_figures}/{experiment_name}.tiff', data_array)
    plt.show()

    return(0)

In [ ]:
def return_data_and_experiment_numbers(region_name, leads,train_start, train_end, val_start, val_end, test_start, test_end,day_num):
        
    obs_final_train,obs_final_validation,obs_final_testing = verifications.open_obs_for_verification(region_name, leads,train_start, train_end, val_start, val_end, test_start, test_end)

    obs_final_testing = obs_final_testing.sel(L=day_num)
    '''Now that we have the observations, we need to loop through each of the testing files and permutate them'''
    
    save_permutation_figures = f'Outputs/permutation_data/{region_name}'
    os.system(f'mkdir -p {save_permutation_figures}')

    #Get the files from the correct location (we need to loop through each of the models.
    # exps = sorted(glob(f'checkpoints/{region_name}/Wk{lead}/*'))
    exps = sorted(glob(f'Data/model_npy_inputs/{region_name}/Wk{lead}_EX_input_data/*testing*'))

    exps = [i for i in exps if 'RZSM' in i]
    exps = [i for i in exps if 'XGBOOST' not in i]

    #Make a list of the experiment names. We only need to grab a single file since they all have the same data
    '''We are going to seperate by hybrid and obs.driven. So remove EX0 and EX13'''
    EX_list = [f'EX{i}' for i in range(26)]
    EX_list = EX_list+['EX27']+['EX28']
    EX_list = [i for i in EX_list if 'EX0' not in i ]
    EX_list = [i for i in EX_list if 'EX13' not in i ]
    # EX_list = [i for i in EX_list if 'EX10' not in i ]

    return(obs_final_train,obs_final_validation,obs_final_testing,EX_list)

In [ ]:
def return_the_experiment_input(exps,file,lead):
    '''First find the experiment name within the file'''
    ex_num = file.split(f'Wk{lead}_')[-1].split('_')[0]

    '''Now loop through exps to grab the correct input'''
    correct_file = [i for i in exps if ex_num in i]
    correct_file = [i for i in correct_file if 'XGBOOST' not in i]

    return(np.load(correct_file[0]),ex_num)

In [ ]:
def permutation_test_by_lead_save_MAE_RMSE(lead, test_year):
    #test 
    # lead = 1
    # test_year=2019

    day_num = (lead*7) -1
    
    file_path = f'Outputs/permutation_tests/mae_rmse_results'
    file_rmse_save = f'{file_path}/Wk{lead}_rmse_vals.pkl'
    file_mae_save = f'{file_path}/Wk{lead}_mae_vals.pkl'
    complete_rmse = f'{file_path}/Wk{lead}_rmse_complete.pkl'
    complete_mae = f'{file_path}/Wk{lead}_mae_complete.pkl'
    
    os.system(f'mkdir -p {file_path}')

    rmse_output, mae_output, rmse_complete, mae_complete = putils.return_rmse_and_mae_pickle_files(file_rmse_save, file_mae_save, complete_mae, complete_rmse)

    ###########################################################################

    leads = [6,13,20,27,34]

    obs_final_train,obs_final_validation,obs_final_testing,EX_list = return_data_and_experiment_numbers(region_name, leads,train_start, train_end, val_start, val_end, test_start, test_end,day_num)

    obs_final_testing_arr = obs_final_testing[putils.xarray_varname(obs_final_testing)].to_numpy()
    '''These are the actual experiment names below'''
    BC,OBS,HYB = verifications.return_experiment_colors_and_names()

    '''Loop through each model checkpoint'''
    checkpt_dir = f'checkpoints/{region_name}/Wk{lead}'
    file_list = sorted(glob(f'{checkpt_dir}/*regular*'))
    file_list = [i for i in file_list if '.' not in i] 
    file_list = [i for i in file_list if '/checkpoint' not in i]
    file_list = [i for i in file_list if 'ECMWF' not in i] #don't do ECMWF. We haven't coded it with different varabiles yet
    file_list = [i for i in file_list if '2012' not in i]

    ##########################################################################
    if test_year == 2019:
        file_list = [i for i in file_list if '2012' not in i]
    else:
        file_list = [i for i in file_list if str(pd.to_datetime(test_end).year) in i]

    '''Experiment saved inputs'''
    exps = sorted(glob(f'Data/model_npy_inputs/{region_name}/Wk{lead}_EX_input_data/*testing*'))
    exps = [i for i in exps if 'XGBOOST' not in i]
    exps = [i for i in exps if 'mean' not in i]

    print(f'Working on file list {file_list}')

    #For masking
    RZSM_train_obs, RZSM_validation_obs = pred.return_masking_objects_for_RZSM(input_directory,final_testing_year)
    mask_zero = RZSM_validation_obs.squeeze()
    
    for file in file_list:
        print(f'Working on file experiment {file}')
        # break
        try:
            testing_input,ex_num = return_the_experiment_input(exps,file,lead)
            channel_list = f.load_channel_list_permutation(ex_num, lead)
            dont_continue=False
        except IndexError:
            dont_continue = True

        try:
            model = load_model(file,compile=False)
            dont_continue = False
        except OSError:
            dont_continue = True # No model information found
            
        if dont_continue:
            pass
        else:
            if (len(channel_list) == testing_input.shape[-1]):

                model_name = file.split('/')[-1].split('_testing')[0]
                
                #These will contain the average MAE and RMSE across CONUS to plot
                save_for_plot_mae = {}
                save_for_plot_rmse = {}

                continue_ = False
                
                for idx,channel in enumerate(channel_list):
                    # break
                    '''Check if we have already completed it'''
                    unit_test = putils.check_if_already_completed_permuatation(rmse_complete, mae_complete, ex_num,  OBS, HYB, channel, model_name)

                    '''Check if there is a .csv file already saved'''
                    
                    if unit_test == 'Not-Completed':
                        # break
                        print(f'\nPermutating channel {channel}\n')
    
                        new_input_with_noise,reforecast_nan,var_noise_min,var_noise_max,var_ = f.load_min_max_files_and_rescale_data(testing_input,channel,idx,file,region_name,day_num,test_year,lead)
    
                        try:
                            prediction_ = np.array(model.predict(new_input_with_noise))
                            prediction_.shape
                            
                            '''Just choose the very last prediction made'''
                            prediction_ = prediction_[-1,:,:,:,0]
        
                            yhat = verifications.reverse_min_max_scaling_for_permutations(prediction_,region_name,day_num,'GEFSv12',test_year,'soilw_bgrnd')
            
                            yhat = np.where(np.isnan(reforecast_nan),np.nan,yhat)
                            yhat = np.where(mask_zero == 0, np.nan, yhat)
                            # yhat = np.where(np.isnan(reforecast_nan),np.nan,yhat.squeeze())
                            yhat = np.reshape(yhat,(yhat.shape[0]//11,11,yhat.shape[1],yhat.shape[2]))
                
                            RZSM_mae =np.nanmean(np.abs(obs_final_testing_arr -  yhat),axis=(0,1))
                
                            RZSM_rmse = np.nanmean((obs_final_testing_arr -  yhat)**2,axis=(0,1))
    
                            save_for_plot_rmse[channel] = np.nanmean(RZSM_rmse)
                            save_for_plot_mae[channel] = np.nanmean(RZSM_mae)
                        
                            '''Now add to the dictionary for RMSE'''
                            if ex_num in OBS:
                                try:
                                    # if output['OBS'][channel]['Value']
                                    rmse_output['OBS'][channel]['Value'] = rmse_output['OBS'][channel]['Value'] + RZSM_rmse
                                    rmse_output['OBS'][channel]['Num_experiments'] = rmse_output['OBS'][channel]['Num_experiments'] + 1
                                    rmse_complete['OBS'][model_name].append(channel)
                                except KeyError:
                                    rmse_output['OBS'][channel] = {'Num_experiments':1, 'Value':RZSM_rmse}
                                    rmse_complete = putils.append_to_complete_list(rmse_complete, model_name, channel, 'OBS')
                                    
                            elif ex_num in HYB:
                                try:
                                    rmse_output['HYB'][channel]['Value'] = rmse_output['HYB'][channel]['Value'] + RZSM_rmse
                                    rmse_output['HYB'][channel]['Num_experiments'] = rmse_output['HYB'][channel]['Num_experiments'] + 1
                                    rmse_complete['HYB'][model_name].append(channel)
                                except KeyError:
                                    rmse_output['HYB'][channel] = {'Num_experiments':1, 'Value':RZSM_rmse}
                                    rmse_complete = putils.append_to_complete_list(rmse_complete, model_name, channel, 'HYB')

                                    
                            '''Now add to the dictionary for MAE only'''
                            if ex_num in OBS:
                                try:
                                    mae_output['OBS'][channel]['Value'] = mae_output['OBS'][channel]['Value'] + RZSM_mae
                                    mae_output['OBS'][channel]['Num_experiments'] = mae_output['OBS'][channel]['Num_experiments'] + 1
                                    mae_complete['OBS'][model_name].append(channel)
                                except KeyError:
                                    mae_output['OBS'][channel] = {'Num_experiments':1, 'Value':RZSM_mae}
                                    mae_complete = putils.append_to_complete_list(mae_complete, model_name, channel, 'OBS')

                            elif ex_num in HYB:
                                try:
                                    mae_output['HYB'][channel]['Value'] = mae_output['HYB'][channel]['Value'] + RZSM_mae
                                    mae_output['HYB'][channel]['Num_experiments'] = mae_output['HYB'][channel]['Num_experiments'] + 1
                                    mae_complete['HYB'][model_name].append(channel)
                                except KeyError:
                                    mae_output['HYB'][channel] = {'Num_experiments':1, 'Value':RZSM_mae}
                                    mae_complete = putils.append_to_complete_list(mae_complete, model_name, channel, 'HYB')

    
                            continue_ = True
                        
                        except ValueError:
                            continue_ = False
                            pass
                        
                if continue_:
                    plot_barplot_mae_rmse(save_for_plot_mae, save_for_plot_rmse, lead, model_name)
                    
            
        '''Saves after each file is completed'''
        with open(file_rmse_save, 'wb') as file:
            pickle.dump(rmse_output, file)
    
        with open(file_mae_save, 'wb') as file:
            pickle.dump(mae_output, file)
        
        with open(complete_rmse, 'wb') as file:
            pickle.dump(rmse_complete, file)
    
        with open(complete_mae, 'wb') as file:
            pickle.dump(mae_complete, file)

    return(f'Completed week {lead}')

In [ ]:
# '''Permutation test'''
# for lead in [1,2,3,4]:
#     permutation_test_by_lead_save_MAE_RMSE(lead=lead, test_year=2019) 

In [ ]:
def permutation_test_save_csv(lead,experiment,train,val,test):
    save_permutation_figures = f'Outputs/permutation_tests/Wk_{lead}'
    os.system(f'mkdir -p {save_permutation_figures}')
    
    day_num = (lead*7)-1
    
    num_lags_obs_RZSM=experiment['num_lags_obs_RZSM']
    include_lags_obs_pwat_spfh_tmax=experiment['include_lags_obs_pwat_spfh_tmax']
    include_reforecast_or_not=experiment['include_reforecast_or_not']
    addtl_experiment = experiment['addtl_experiment']
    experiment_test = experiment['experiment_test']

    experiment_name = CE.return_experiment_name(include_reforecast_or_not,num_lags_obs_RZSM,include_lags_obs_pwat_spfh_tmax,addtl_experiment,experiment_test)
    experiment_name_out = f'{experiment_name}_RZSM'
    
    save_csv_path = f'{save_permutation_figures}/{experiment_name_out}.csv'    
    obs_final_testing_arr = test.RZSM.values

    obs_final_testing_arr = np.reshape(obs_final_testing_arr,(1144,48,96))
    
    ref_data = f'Data/model_npy_inputs/{region_name}/Wk{lead}_EX_input_data/{experiment_name_out}_testing_input.npy'
    
    if os.path.exists('redoooo.nc'):
    # if os.path.exists(save_csv_path):
        print(f'Already saved the csv file into path {save_csv_path}')
    elif os.path.exists(ref_data):
        reg_name = f'{experiment_name_out.split("_")[0]}_regular_RZSM'

        print(f'\n\n Starting permutation test with experiment {reg_name} \n\n')
        
        reforecast_testing_input = np.load(ref_data)
        
        #Load model 
        try:
            model = load_model(f'checkpoints/{region_name}/Wk{lead}/Wk{lead}_{reg_name}',compile=False) #don't need the custom loss function for predictions
            channel_list = f.load_channel_list_permutation(experiment_name_out, lead)
        except OSError:
            model_name = f'checkpoints/{region_name}/Wk{lead}/Wk{lead}_{experiment_name_out}'
            model = load_model(model_name,compile=False) #don't need the custom loss function for predictions
            channel_list = f.load_channel_list_permutation(experiment_name_out, lead)

        
        out_dict = {}
        for idx,channel in enumerate(channel_list):
            print(f'\nPermutating channel {channel}\n')
            # break
            '''Take the input, reverse back to anomaly, find mean and std, add gaussian noise according to the mean and std, get min max, then rescale to min max'''
            new_input_with_noise,reforecast_nan,var_noise_min,var_noise_max,var_ = f.load_min_max_files_and_rescale_data(reforecast_testing_input,channel,idx,'GEFSv12',region_name,day_num,test_year,lead)
            #The output is now min max scaled

            pred_ = np.array(model.predict(new_input_with_noise))
            '''Just choose the very last prediction made'''
            pred_ = pred_[-1,:,:,:,0]

            #Reverse the min max scaling from prediction back to anomaly
            # pred_ = pred_ *(var_noise_max-var_noise_min)+var_noise_min 
            pred_ = verifications.reverse_min_max_scaling(pred_, region_name, day_num, 'GEFSv12',2019) #We only want the last channel
            '''rescale back to anomaly with the min and max from the noisy dataset'''
            #Now data is in the anomaly form

            
            pred_ = np.where(np.isnan(reforecast_nan),np.nan,pred_)
            pred_ = np.where(np.isnan(obs_final_testing_arr), np.nan, pred_) #observation mask
            # yhat = np.where(np.isnan(reforecast_nan),np.nan,yhat.squeeze())
            # pred_ = np.reshape(pred_,(pred_.shape[0]//11,11,pred_.shape[1],pred_.shape[2]))

            RZSM_mae =np.nanmean(np.abs(obs_final_testing_arr -  pred_))
            # RZSM_rmse = np.nanmean((obs_final_testing_arr -  prediction_)**2)

            out_dict[f'{channel}'] = np.nanmean(RZSM_mae)
            print(f'RZSM MAE of channel {channel} is {np.nanmean(RZSM_mae)}')
            contn = True
            # except ValueError:
            #     print('We could not complete this file, there is something wrong with the shape of the inputs''')
            #     contn = False
            #     break
                
        if contn:
            csv_dict = {}
            #Experiment_name
            # Remove 'E', 'X', and '_RZSM' from the string
            result_string = experiment_name_out.replace('E', '').replace('X', '').replace('_RZSM', '')
        
            csv_dict['Experiment'] = int(result_string)
            for i in list(out_dict.keys()):
                csv_dict[i] = out_dict[i]
                
            #Write to csv file for later processing
            with open(save_csv_path, 'w') as csvfile:
                writer = csv.DictWriter(csvfile, fieldnames=list(csv_dict.keys()))
                writer.writeheader()
                writer.writerows([csv_dict])
    
        # create_plot_permutation(setup_plot_permutation(lead,out_dict,RZSM_or_Tmax_or_both),lead,experiment_name_out,save_permutation_figures,max_RZSM_value,max_tmax_value,RZSM_or_Tmax_or_both)


In [ ]:

    #Testing the input layer for issues
#         for experiment in range(0,13):
#             print(f'Loading EX{experiment}')
#             model = load_model(f'checkpoints/Wk_{lead}/Wk{lead}_EX{experiment}',compile=False) #don't need the custom loss function for predictions

#             summary_str = None
#             with StringIO() as buffer, redirect_stdout(buffer):
#                 model.summary()
#                 summary_str = buffer.getvalue()

#             # Print the head of the summary
#             head_lines = 5  # Adjust the number of lines you want to print
#             print("\n".join(summary_str.splitlines()[:head_lines]))

In [ ]:
#Load data, setup experiments to permutate

all_exps = [EX0,EX1,EX2,EX3,EX4,EX5,EX6,EX7,EX8,EX9,EX10,EX11,EX12,EX13,EX14,EX15,EX16,EX17,EX18,EX19,EX20,EX21,EX22,EX23,EX24,EX25,EX27,EX28]
obs_final_train,obs_final_validation,obs_final_testing = verifications.open_obs_for_verification(region_name, [6,13,20,27],train_start, train_end, val_start, val_end, test_start, test_end)



In [ ]:
sub_exps = [EX13,EX14,EX15,EX16,EX17,EX18,EX19,EX20,EX22,EX23,EX24,EX25,EX27,EX28]

redo_2thru_4 = [EX1,EX2,EX3,EX4,EX5,EX6,EX7,EX8,EX9,EX10,EX11,]
sub_exps = [EX29]
for lead in [1,2,3,4]:
    day_num = (lead*7)-1
    train = obs_final_train.sel(L=day_num)
    val = obs_final_validation.sel(L=day_num)
    test = obs_final_testing.sel(L=day_num)
    # for experiment in all_exps:
    for experiment in sub_exps:
        permutation_test_save_csv(lead,experiment,train, val, test)

In [ ]:
# def permutation_test_by_lead_by_experiment(lead, test_year, experiment_number):
#     #test 
#     # lead = 1
#     # test_year=2019
    
#     day_num = (lead*7) -1

#     try:
#         with open(file_rmse_save, "rb") as f:
#             rmse_output = pickle.load(f)
    
#         with open(file_mae_save, "rb") as f:
#             mae_output = pickle.load(f)    
#     except FileNotFoundError:
#         rmse_output = {}
#         rmse_output['OBS'] = {}
#         rmse_output['HYB'] = {}
    
#         mae_output = {}
#         mae_output['OBS'] = {}
#         mae_output['HYB'] = {}

    
#     leads = [6,13,20,27,34]

#     obs_final_train,obs_final_validation,obs_final_testing,EX_list = return_data_and_experiment_numbers(region_name, leads,train_start, train_end, val_start, val_end, test_start, test_end,day_num)

#     obs_final_testing_arr = obs_final_testing[putils.xarray_varname(obs_final_testing)].to_numpy()
#     '''These are the actual experiment names below'''
#     BC,OBS,HYB = verifications.return_experiment_colors_and_names()


#     '''Loop through each model checkpoint'''
#     checkpt_dir = f'checkpoints/{region_name}/Wk{lead}'
#     file_list = sorted(glob(f'{checkpt_dir}/*{experiment_number}_regular*'))
  
    
#     if test_year == 2019:
#         file_list = [i for i in file_list if '2012' not in i]
#     else:
#         file_list = [i for i in file_list if str(pd.to_datetime(test_end).year) in i]

#     '''Experiment saved inputs'''
#     exps = sorted(glob(f'Data/model_npy_inputs/{region_name}/Wk{lead}_EX_input_data/*testing*'))
#     exps = [i for i in exps if 'XGBOOST' not in i]
#     exps = [i for i in exps if 'mean' not in i]

#     print(f'Working on file list {file_list}')

#     #For masking
#     RZSM_train_obs, RZSM_validation_obs = pred.return_masking_objects_for_RZSM(input_directory,final_testing_year)
#     mask_zero = RZSM_validation_obs.squeeze()
    
#     for file in file_list:
#         print(f'Working on file experiment {file}')
#         try:
#             testing_input,ex_num = return_the_experiment_input(exps,file,lead)
#             channel_list = f.load_channel_list_permutation(ex_num, lead)
#             dont_continue=False
            
#         except IndexError:
#             dont_continue = True

#         if dont_continue:
#             pass
#         else:
#             if len(channel_list) == testing_input.shape[-1]:
#                 model = load_model(file,compile=False) 
#                 model_name = file.split('/')[-1].split('_testing')[0]
                
#                 #These will contain the average MAE and RMSE across CONUS to plot
#                 save_for_plot_mae = {}
#                 save_for_plot_rmse = {}
                
#                 for idx,channel in enumerate(channel_list):
#                     '''Now add to the dictionary'''
#                     if ex_num in OBS:
#                         try:
#                             rmse_output['OBS'][channel]['Value'] = rmse_output['OBS'][channel]['Value'] + RZSM_rmse
#                             rmse_output['OBS'][channel]['Num_experiments'] = rmse_output['OBS'][channel]['Num_experiments'] + 1
#                         except KeyError:
#                             rmse_output['OBS'][channel] = {'Num_experiments':1, 'Value':RZSM_rmse}
#                     elif ex_num in HYB:
                        
                
#                     # idx,channel = 6, 'pwat_obs_lag-1'
#                     # idx, channel = 0, 'RZSM_obs_lag-1'
                    
#                     new_input_with_noise,reforecast_nan,var_noise_min,var_noise_max,var_ = f.load_min_max_files_and_rescale_data(testing_input,channel,idx,file,region_name,day_num,test_year,lead)

#                     try:
#                         prediction_ = np.array(model.predict(new_input_with_noise))
#                         prediction_.shape
                        
#                         '''Just choose the very last prediction made'''
#                         prediction_ = prediction_[-1,:,:,:,0]
        
#                         yhat = verifications.reverse_min_max_scaling_for_permutations(prediction_,region_name,day_num,'GEFSv12',test_year,'soilw_bgrnd')
        
#                         yhat = np.where(np.isnan(reforecast_nan),np.nan,yhat)
#                         yhat = np.where(mask_zero == 0, np.nan, yhat)
#                         # yhat = np.where(np.isnan(reforecast_nan),np.nan,yhat.squeeze())
#                         yhat = np.reshape(yhat,(yhat.shape[0]//11,11,yhat.shape[1],yhat.shape[2]))
            
#                         RZSM_mae =np.nanmean(np.abs(obs_final_testing_arr -  yhat),axis=(0,1))
            
#                         RZSM_rmse = np.nanmean((obs_final_testing_arr -  yhat)**2,axis=(0,1))
    
#                         save_for_plot_rmse[channel] = np.nanmean(RZSM_rmse)
#                         save_for_plot_mae[channel] = np.nanmean(RZSM_mae)
                        
#                         '''Now add to the dictionary'''
#                         if ex_num in OBS:
#                             try:
#                                 rmse_output['OBS'][channel]['Value'] = rmse_output['OBS'][channel]['Value'] + RZSM_rmse
#                                 rmse_output['OBS'][channel]['Num_experiments'] = rmse_output['OBS'][channel]['Num_experiments'] + 1
#                             except KeyError:
#                                 rmse_output['OBS'][channel] = {'Num_experiments':1, 'Value':RZSM_rmse}
#                         elif ex_num in HYB:
#                             try:
#                                 rmse_output['HYB'][channel]['Value'] = rmse_output['HYB'][channel]['Value'] + RZSM_rmse
#                                 rmse_output['HYB'][channel]['Num_experiments'] = rmse_output['HYB'][channel]['Num_experiments'] + 1
#                             except KeyError:
#                                 rmse_output['HYB'][channel] = {'Num_experiments':1, 'Value':RZSM_rmse}
    
    
#                         '''Now add to the dictionary'''
#                         if ex_num in HYB:
#                             try:
#                                 mae_output['OBS'][channel]['Value'] = mae_output['OBS'][channel]['Value'] + RZSM_mae
#                                 mae_output['OBS'][channel]['Num_experiments'] = mae_output['OBS'][channel]['Num_experiments'] + 1
#                             except KeyError:
#                                 mae_output['OBS'][channel] = {'Num_experiments':1, 'Value':RZSM_mae}
#                         elif ex_num in HYB:
#                             try:
#                                 mae_output['HYB'][channel]['Value'] = mae_output['HYB'][channel]['Value'] + RZSM_mae
#                                 mae_output['HYB'][channel]['Num_experiments'] = mae_output['HYB'][channel]['Num_experiments'] + 1
#                             except KeyError:
#                                 mae_output['HYB'][channel] = {'Num_experiments':1, 'Value':RZSM_mae}

#                         continue_ = True
                    
            
                    
#                     except ValueError:
#                         continue_ = False
#                         pass
                        
#                 if continue_:
#                     plot_barplot_mae_rmse(save_for_plot_mae, save_for_plot_rmse, lead, model_name)
                    

#     return(f'Completed week {lead}')

In [ ]:

t

In [ ]:
sub_exps= [EX1,EX2,EX3,EX4,EX5,EX6,EX7,EX8,EX9,EX10,EX11,EX14,EX15,EX18,EX19,EX20,EX22,EX23,EX24,EX25,EX27]

In [ ]:
all_exps= [EX3,EX4,EX11,EX16,EX17,EX18,EX19,EX20,EX21]
all_exps = [EX11,EX16]
for experiment in all_exps:
    run_model_and_prediction(experiment)
t

In [ ]:
all_exps= [EX0,EX1,EX2,EX3,EX4,EX5,EX6,EX7,EX8,EX9,EX10,EX11,EX13,EX14,EX15,EX16,EX17,EX18,EX19,EX20,EX21,EX22,EX23,EX24,EX25,EX27,EX28]
all_exps= [EX3,EX4,EX11,EX16,EX17,EX18,EX19,EX20,EX21]

# for lead in [1,2,3,4]:
#     permutation_test_by_lead(lead=lead, test_year=2019) 

good= [EX0,EX1,EX2,EX5,EX6,EX7,EX8,EX9,EX10,EX13,EX14,EX15,EX22,EX23,EX24,EX25,EX27,EX28]


In [ ]:
missing_week4 = [EX11,EX16,EX17,EX21,EX28]

for experiment in good:
    run_model_and_prediction(experiment)

t

In [ ]:
all_exps= [EX1]

for experiment in all_exps:
    run_model_and_prediction(experiment)
t

In [ ]:
all_exps= [EX0,EX1,EX2,EX3,EX4,EX5,EX6,EX7,EX8,EX9,EX10,EX11,EX13,EX14,EX15,EX16,EX17,EX18,EX19,EX20,EX21,EX22,EX23,EX24,EX25,EX27,EX28]

'''We are not testing EX12 or EX26'''
for idx,experiment in enumerate(all_exps):
    run_model_and_prediction(experiment)

In [ ]:
'''Run the single prediction tests'''
        
%load_ext autoreload 
%autoreload 2
%reload_ext autoreload
    
testing_scenarios = ['dense', 'regular', 'transformer', 'super_pixel', 'attention', 'denseLarge', 'basic']

for testing_scenario in testing_scenarios:
    #removed EX12 and EX26
    all_exps= [EX0,EX1,EX2,EX3,EX4,EX5,EX6,EX7,EX8,EX9,EX10,EX11,EX13,EX14,EX15,EX16,EX17,EX18,EX19,EX20,EX21,EX22,EX23,EX24,EX25,EX27,EX28]

    
    global save_loss_name, loss_fn
    
    if testing_scenario == 'dense':
        import modelRzsmDenseRelu as UNETRzsm
        
        save_loss_name = 'denseLoss'
        loss_fn = losses.crps2d_tf_dense_test2
        
    elif testing_scenario == 'regular':
        import modelRzsmReluExtraConv as UNETRzsm
    
        save_loss_name = 'regular'
        loss_fn = losses.crps2d_tf
    
    elif testing_scenario == 'transformer':
        import modelRzsmReluTransformer as UNETRzsm
        
        save_loss_name = 'transformer'
        loss_fn = losses.crps2d_tf_dense_test2
    
    elif testing_scenario == 'super_pixel':
        import modelRzsmReluPixelSuper as UNETRzsm
        
        save_loss_name = 'supPixel'
        loss_fn = losses.crps2d_tf_dense_test2
    
    elif testing_scenario == 'attention':
        import modelRzsmReluAttention as UNETRzsm
        
        save_loss_name = 'attention'
        loss_fn = losses.crps2d_tf_dense_test2
        
    elif testing_scenario == 'None':
        import modelRzsm2DdropoutRelu as UNETRzsm
        
        save_loss_name = ''
        loss_fn = losses.crps2d_tf
    
    elif testing_scenario == 'denseLarge':
        import modelRzsmReluExtraConv as UNETRzsm
    
        save_loss_name = 'denseLarge'
        loss_fn = losses.crps2d_tf_dense_test2
        
    elif testing_scenario == 'basic':
        import basicUNET as UNETRzsm
    
        save_loss_name = testing_scenario
        loss_fn = losses.crps2d_tf

    for idx,experiment in enumerate(all_exps):
        run_model_and_prediction(experiment)

In [ ]:
all_exps= [EX0,EX1,EX2,EX3,EX4,EX5,EX6,EX7,EX8,EX9,EX10,EX11,EX12,EX13,EX14,EX15,EX16,EX17,EX18,EX19,EX20,EX21,EX22,EX23,EX24,EX25,EX27,EX28]
for idx,experiment in enumerate(all_exps):
    run_model_and_prediction(experiment)

In [ ]:
for idx,experiment in enumerate(all_exps):
    run_model_and_prediction(experiment)

In [ ]:
# for experiment_ in [EX14,EX15,EX16,EX17,EX18,EX19,EX20,EX21,EX22,EX23,EX24,EX25,EX27,EX28]:
#     global testing_scenario
#     testing_scenario = 'None'
#     run_model_and_prediction(experiment_)
#     for test in ['dense', 'regular', 'transformer', 'super_pixel', 'attention', 'denseLarge', 'basic', None]:
        

In [ ]:
# run_model_and_prediction(EX27)

# After all the experiments are completed, now do a permutation test
## Save the results for each type (observation or hybrid driven) only

In [ ]:
for lead in [1,2]:
    permutation_test_by_lead(lead=lead, test_year=2019)

In [ ]:
for lead in [1]:
    permutation_test_by_lead_by_experiment(lead=lead, test_year=2019, experiment_number = 'EX1')

In [ ]:
experiment=EX29
#for testing
lead,lead_week = 3,3

experiment['region_name']
lead = lead_week
num_lags_obs_RZSM=experiment['num_lags_obs_RZSM']
include_lags_obs_pwat_spfh_tmax=experiment['include_lags_obs_pwat_spfh_tmax']
include_reforecast_or_not=experiment['include_reforecast_or_not']
addtl_experiment = experiment['addtl_experiment']
experiment_test = experiment['experiment_test']

epochs = 150
batch_size = 66
initial_learning_rate=0.0001
beta_1 = 0.9
shuffle=False
patience=10
kernel_norm = None
deep_supervision = True
number_of_UNET_backbone_max_pool=4


# def return_fully_autoregressive_EX26(lead):

    #We can manually select the best models and create a week N estimate based on them (this implies that all the information from the observations is contained in the first 
    #Set of leads

    best_models = []
    
    if lead == 2:
        previous_model = ['EX10_RZSM','EX20_RZSM']
    elif lead == 3:
        previous_model = ['EX10_RZSM','EX20_RZSM','EX20_RZSM']
    elif lead == 4:
        previous_model = ['EX10_RZSM','EX20_RZSM','EX20_RZSM','EX20_RZSM']
    elif lead == 5:
        previous_model = ['EX10_RZSM','EX20_RZSM','EX20_RZSM','EX20_RZSM','EX20_RZSM']

    return(previous_model)
